In [1]:
from copy import deepcopy
import os
import sys
from datetime import datetime

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

sys.path.append(os.path.dirname(sys.path[0]))
plt.style.use('seaborn')

In [3]:
from scripts.configurations.converters import PDBConverter
from scripts.configurations.static_system import StaticSystem
import scripts.constants as con
from scripts.exp_constructor import EXPConstructor

In [4]:
#   #####################################################################################   #
#   ############################### IMPORT WHAT IS NEEDED ###############################   #

from fullrmc.Engine import Engine
from fullrmc.Constraints.PairDistributionConstraints import PairDistributionConstraint
from fullrmc.Constraints.DistanceConstraints import InterMolecularDistanceConstraint
from fullrmc.Constraints.BondConstraints import BondConstraint
from fullrmc.Constraints.AngleConstraints import BondsAngleConstraint
from fullrmc.Constraints.ImproperAngleConstraints import ImproperAngleConstraint
from fullrmc.Core.MoveGenerator import MoveGeneratorCollector
from fullrmc.Generators.Translations import TranslationGenerator, TranslationAlongSymmetryAxisGenerator
from fullrmc.Generators.Rotations import RotationGenerator, RotationAboutSymmetryAxisGenerator

In [5]:
CURRENT_DATA_NORMAL = '2022-03-13_prepared_1.3_normal'
PLOT_FILENAME_POSTFIX_NORMAL = 'rmc_minimization'

PATH_TO_CURRENT_DATA_NORMAL = os.path.join(con.PATH_TO_DATA, CURRENT_DATA_NORMAL)
PATH_TO_PLOTS = os.path.join(con.PATH_TO_PLOTS, CURRENT_DATA_NORMAL)

PATH_TO_CURRENT_DATA_NORMAL

'D:\\albert\\programming\\git_reps\\molecular_dynamics\\data\\2022-03-13_prepared_1.3_normal'

In [6]:
cell_dimensions = 12.25 * np.ones(3)

In [7]:
test_sample = np.load(os.path.join(PATH_TO_CURRENT_DATA_NORMAL, 'test_sample.npy'))
cooled_config = StaticSystem(
    positions=np.load(os.path.join(PATH_TO_CURRENT_DATA_NORMAL, 'cooled_1.npy')),
    cell_dimensions=cell_dimensions,
    is_pbc_applied=False,
    time=0,
)
test_sample.shape, cooled_config.positions.shape

((1, 612), (1372, 3))

In [8]:
rdf_data = pd.DataFrame(columns=['radius', 'rdf'], index=np.arange(test_sample.size))
rdf_data['radius'] = (rdf_data.index + 1) * 0.01
rdf_data['rdf'] = test_sample.T
pdfData = os.path.join(PATH_TO_CURRENT_DATA_NORMAL, 'test_sample.exp')
EXPConstructor(dataframe=rdf_data, exp_file_name=pdfData.removesuffix('.exp')).save()
rdf_data

,radius,rdf
0,0.01,0.000000
1,0.02,0.000000
2,0.03,0.000000
3,0.04,0.000000
4,0.05,0.000000
...,...,...
607,6.08,1.001271
608,6.09,1.001697
609,6.10,1.001940
610,6.11,1.002143


In [9]:
pdbStructure = os.path.join(PATH_TO_CURRENT_DATA_NORMAL, 'cooled_config.pdb')

with open(pdbStructure, mode='w', encoding='utf8') as file:
    string = PDBConverter().get_string(
        configuration=cooled_config,
    )
    file.write(string)

In [10]:
#   #####################################################################################   #
#   ############################# DECLARE USEFUL VARIABLES ##############################   #
enginePath = os.path.join(PATH_TO_CURRENT_DATA_NORMAL, "thf_engine.rmc")

In [11]:
#   #####################################################################################   #
#   ############################## CREATE STOCHASTIC ENGINE #############################   #

ENGINE = Engine(path=None)
# if engine is not created and saved
if not ENGINE.is_engine(enginePath):
    # create and initialize engine
    ENGINE = Engine(path=enginePath, freshStart=True)
    ENGINE.set_pdb(pdbStructure)

    # re-set structure boundary conditions
    ENGINE.set_boundary_conditions(np.array([12.25]))

    # create and add pair distribution constraint to the engine
    PDF_CONSTRAINT = PairDistributionConstraint(experimentalData=pdfData, weighting="atomicNumber")
    ENGINE.add_constraints([PDF_CONSTRAINT])

    # create and add intermolecular distances constraint to the engine
    EMD_CONSTRAINT = InterMolecularDistanceConstraint(defaultDistance=1.5)
    ENGINE.add_constraints([EMD_CONSTRAINT])

    # initialize constraints data
    ENGINE.initialize_used_constraints()

    # save engine
    ENGINE.save()
# if engine is created and saved, it can be simply loaded.
else:
    ENGINE = ENGINE.load(enginePath)
    # unpack constraints before fitting in case tweaking is needed
    PDF_CONSTRAINT, EMD_CONSTRAINT = ENGINE.constraints

C:\Users\albert\Envs\md_env\lib\site-packages\pdbparser\Utilities\BoundaryConditions.py:275: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  [0, 0, vectorsArray] ], dtype = np.float )
C:\Users\albert\Envs\md_env\lib\site-packages\pdbparser\Utilities\BoundaryConditions.py:275: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  [0, 0, vectorsArray

2022-04-23 00:20:17 - fullrmc <INFO> Given constraints will be added to all normal frames in engine
2022-04-23 00:20:17 - fullrmc <INFO> Given constraints will be added to all normal frames in engine
2022-04-23 00:20:18 - fullrmc <INFO> @0 Initializing constraint data 'PairDistributionConstraint'
2022-04-23 00:20:18 - fullrmc <INFO> @0 Initializing constraint data 'InterMolecularDistanceConstraint'
2022-04-23 00:20:18 - fullrmc <INFO> Saving Engine and frame 0 data... DON'T INTERRUPT
2022-04-23 00:20:33 - fullrmc <INFO> Engine and frame 0 data saved successfuly to 'D:\albert\programming\git_reps\molecular_dynamics\data\2022-03-13_prepared_1.3_normal\thf_engine.rmc'


In [12]:
#   #####################################################################################   #
#   ########################### RUN ATOMIC STOCHASTIC FITTING ###########################   #
# set groups as atoms. By default when the engine is constructed, all groups are single atoms.
ENGINE.set_groups_as_atoms()
ENGINE.run(numberOfSteps=100000, saveFrequency=1000)

2022-04-23 00:20:33 - fullrmc <INFO> Engine @0 started 100000 steps, total standard error is: 62576.558594
2022-04-23 00:20:34 - fullrmc <INFO> @0 Gen:5 - Tr:2(40.000%) - Acc:1(20.000%) - Rem:0(0.000%) - Err:62482.957031
2022-04-23 00:20:34 - fullrmc <INFO> @0 Gen:6 - Tr:3(50.000%) - Acc:2(33.333%) - Rem:0(0.000%) - Err:62481.277344
2022-04-23 00:20:34 - fullrmc <INFO> @0 Gen:7 - Tr:4(57.143%) - Acc:3(42.857%) - Rem:0(0.000%) - Err:62350.351562
2022-04-23 00:20:34 - fullrmc <INFO> @0 Gen:10 - Tr:6(60.000%) - Acc:4(40.000%) - Rem:0(0.000%) - Err:62337.351562
2022-04-23 00:20:34 - fullrmc <INFO> @0 Gen:11 - Tr:7(63.636%) - Acc:5(45.455%) - Rem:0(0.000%) - Err:62268.929688
2022-04-23 00:20:34 - fullrmc <INFO> @0 Gen:12 - Tr:8(66.667%) - Acc:6(50.000%) - Rem:0(0.000%) - Err:62210.992188
2022-04-23 00:20:34 - fullrmc <INFO> @0 Gen:17 - Tr:10(58.824%) - Acc:7(41.176%) - Rem:0(0.000%) - Err:62131.968750
2022-04-23 00:20:34 - fullrmc <INFO> @0 Gen:18 - Tr:11(61.111%) - Acc:8(44.444%) - Rem:0(0

2022-04-23 00:20:36 - fullrmc <INFO> @0 Gen:113 - Tr:75(66.372%) - Acc:71(62.832%) - Rem:0(0.000%) - Err:56963.304688
2022-04-23 00:20:36 - fullrmc <INFO> @0 Gen:116 - Tr:76(65.517%) - Acc:72(62.069%) - Rem:0(0.000%) - Err:56863.257812
2022-04-23 00:20:36 - fullrmc <INFO> @0 Gen:120 - Tr:77(64.167%) - Acc:73(60.833%) - Rem:0(0.000%) - Err:56861.667969
2022-04-23 00:20:36 - fullrmc <INFO> @0 Gen:121 - Tr:78(64.463%) - Acc:74(61.157%) - Rem:0(0.000%) - Err:56854.460938
2022-04-23 00:20:36 - fullrmc <INFO> @0 Gen:122 - Tr:79(64.754%) - Acc:75(61.475%) - Rem:0(0.000%) - Err:56803.371094
2022-04-23 00:20:36 - fullrmc <INFO> @0 Gen:123 - Tr:80(65.041%) - Acc:76(61.789%) - Rem:0(0.000%) - Err:56692.722656
2022-04-23 00:20:36 - fullrmc <INFO> @0 Gen:128 - Tr:81(63.281%) - Acc:77(60.156%) - Rem:0(0.000%) - Err:56658.398438
2022-04-23 00:20:36 - fullrmc <INFO> @0 Gen:129 - Tr:82(63.566%) - Acc:78(60.465%) - Rem:0(0.000%) - Err:56590.226562
2022-04-23 00:20:36 - fullrmc <INFO> @0 Gen:135 - Tr:84(

2022-04-23 00:20:38 - fullrmc <INFO> @0 Gen:245 - Tr:149(60.816%) - Acc:140(57.143%) - Rem:0(0.000%) - Err:52680.882812
2022-04-23 00:20:38 - fullrmc <INFO> @0 Gen:248 - Tr:150(60.484%) - Acc:141(56.855%) - Rem:0(0.000%) - Err:52663.378906
2022-04-23 00:20:38 - fullrmc <INFO> @0 Gen:249 - Tr:151(60.643%) - Acc:142(57.028%) - Rem:0(0.000%) - Err:52604.386719
2022-04-23 00:20:38 - fullrmc <INFO> @0 Gen:252 - Tr:153(60.714%) - Acc:143(56.746%) - Rem:0(0.000%) - Err:52493.929688
2022-04-23 00:20:38 - fullrmc <INFO> @0 Gen:253 - Tr:154(60.870%) - Acc:144(56.917%) - Rem:0(0.000%) - Err:52442.335938
2022-04-23 00:20:38 - fullrmc <INFO> @0 Gen:254 - Tr:155(61.024%) - Acc:145(57.087%) - Rem:0(0.000%) - Err:52398.441406
2022-04-23 00:20:38 - fullrmc <INFO> @0 Gen:255 - Tr:156(61.176%) - Acc:146(57.255%) - Rem:0(0.000%) - Err:52374.062500
2022-04-23 00:20:38 - fullrmc <INFO> @0 Gen:256 - Tr:157(61.328%) - Acc:147(57.422%) - Rem:0(0.000%) - Err:52239.160156
2022-04-23 00:20:38 - fullrmc <INFO> @0 

2022-04-23 00:20:40 - fullrmc <INFO> @0 Gen:387 - Tr:228(58.915%) - Acc:209(54.005%) - Rem:0(0.000%) - Err:48843.824219
2022-04-23 00:20:40 - fullrmc <INFO> @0 Gen:389 - Tr:229(58.869%) - Acc:210(53.985%) - Rem:0(0.000%) - Err:48779.375000
2022-04-23 00:20:40 - fullrmc <INFO> @0 Gen:390 - Tr:230(58.974%) - Acc:211(54.103%) - Rem:0(0.000%) - Err:48771.335938
2022-04-23 00:20:40 - fullrmc <INFO> @0 Gen:391 - Tr:231(59.079%) - Acc:212(54.220%) - Rem:0(0.000%) - Err:48765.902344
2022-04-23 00:20:41 - fullrmc <INFO> @0 Gen:393 - Tr:232(59.033%) - Acc:213(54.198%) - Rem:0(0.000%) - Err:48664.703125
2022-04-23 00:20:41 - fullrmc <INFO> @0 Gen:395 - Tr:233(58.987%) - Acc:214(54.177%) - Rem:0(0.000%) - Err:48632.261719
2022-04-23 00:20:41 - fullrmc <INFO> @0 Gen:396 - Tr:234(59.091%) - Acc:215(54.293%) - Rem:0(0.000%) - Err:48596.355469
2022-04-23 00:20:41 - fullrmc <INFO> @0 Gen:403 - Tr:237(58.809%) - Acc:216(53.598%) - Rem:0(0.000%) - Err:48476.355469
2022-04-23 00:20:41 - fullrmc <INFO> @0 

2022-04-23 00:20:43 - fullrmc <INFO> @0 Gen:507 - Tr:308(60.750%) - Acc:278(54.832%) - Rem:0(0.000%) - Err:45397.246094
2022-04-23 00:20:43 - fullrmc <INFO> @0 Gen:508 - Tr:309(60.827%) - Acc:279(54.921%) - Rem:0(0.000%) - Err:45384.777344
2022-04-23 00:20:43 - fullrmc <INFO> @0 Gen:510 - Tr:310(60.784%) - Acc:280(54.902%) - Rem:0(0.000%) - Err:45382.148438
2022-04-23 00:20:43 - fullrmc <INFO> @0 Gen:512 - Tr:311(60.742%) - Acc:281(54.883%) - Rem:0(0.000%) - Err:45360.550781
2022-04-23 00:20:43 - fullrmc <INFO> @0 Gen:514 - Tr:312(60.700%) - Acc:282(54.864%) - Rem:0(0.000%) - Err:45252.566406
2022-04-23 00:20:43 - fullrmc <INFO> @0 Gen:515 - Tr:313(60.777%) - Acc:283(54.951%) - Rem:0(0.000%) - Err:45251.078125
2022-04-23 00:20:43 - fullrmc <INFO> @0 Gen:518 - Tr:314(60.618%) - Acc:284(54.826%) - Rem:0(0.000%) - Err:45190.609375
2022-04-23 00:20:43 - fullrmc <INFO> @0 Gen:519 - Tr:315(60.694%) - Acc:285(54.913%) - Rem:0(0.000%) - Err:45112.851562
2022-04-23 00:20:43 - fullrmc <INFO> @0 

...

2022-04-23 00:30:15 - fullrmc <INFO> @0 Gen:78484 - Tr:23152(29.499%) - Acc:5363(6.833%) - Rem:0(0.000%) - Err:3504.722900
2022-04-23 00:30:15 - fullrmc <INFO> @0 Gen:78794 - Tr:23248(29.505%) - Acc:5364(6.808%) - Rem:0(0.000%) - Err:3504.501709
2022-04-23 00:30:16 - fullrmc <INFO> @0 Gen:78849 - Tr:23267(29.508%) - Acc:5365(6.804%) - Rem:0(0.000%) - Err:3503.134033
2022-04-23 00:30:16 - fullrmc <INFO> @0 Gen:78881 - Tr:23279(29.512%) - Acc:5366(6.803%) - Rem:0(0.000%) - Err:3502.533936
2022-04-23 00:30:16 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 00:30:20 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 00:30:20 - fullrmc <INFO> @0 Gen:79009 - Tr:23321(29.517%) - Acc:5367(6.793%) - Rem:0(0.000%) - Err:3501.015137
2022-04-23 00:30:20 - fullrmc <INFO> @0 Gen:79032 - Tr:23331(29.521%) - Acc:5368(6.792%) - Rem:0(0.000%) - Err:3499.283691
2022-04-23 00:30:20 - fullrmc <INFO> @0 Gen:79078 - Tr:23345(29.521%) - Acc:5369(6.789%) - Rem:0(0.000%) - Err:3499.2326

2022-04-23 00:30:44 - fullrmc <INFO> @0 Gen:83602 - Tr:24650(29.485%) - Acc:5424(6.488%) - Rem:0(0.000%) - Err:3463.138184
2022-04-23 00:30:44 - fullrmc <INFO> @0 Gen:83606 - Tr:24653(29.487%) - Acc:5425(6.489%) - Rem:0(0.000%) - Err:3463.111816
2022-04-23 00:30:44 - fullrmc <INFO> @0 Gen:83665 - Tr:24670(29.487%) - Acc:5426(6.485%) - Rem:0(0.000%) - Err:3462.520508
2022-04-23 00:30:44 - fullrmc <INFO> @0 Gen:83680 - Tr:24676(29.489%) - Acc:5427(6.485%) - Rem:0(0.000%) - Err:3461.567383
2022-04-23 00:30:44 - fullrmc <INFO> @0 Gen:83746 - Tr:24695(29.488%) - Acc:5428(6.482%) - Rem:0(0.000%) - Err:3461.268555
2022-04-23 00:30:44 - fullrmc <INFO> @0 Gen:83760 - Tr:24699(29.488%) - Acc:5429(6.482%) - Rem:0(0.000%) - Err:3460.087646
2022-04-23 00:30:44 - fullrmc <INFO> @0 Gen:83822 - Tr:24714(29.484%) - Acc:5430(6.478%) - Rem:0(0.000%) - Err:3459.973145
2022-04-23 00:30:44 - fullrmc <INFO> @0 Gen:83851 - Tr:24721(29.482%) - Acc:5431(6.477%) - Rem:0(0.000%) - Err:3459.806641
2022-04-23 00:30

2022-04-23 00:31:08 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 00:31:12 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 00:31:12 - fullrmc <INFO> @0 Gen:88028 - Tr:25882(29.402%) - Acc:5486(6.232%) - Rem:0(0.000%) - Err:3438.661133
2022-04-23 00:31:12 - fullrmc <INFO> @0 Gen:88059 - Tr:25891(29.402%) - Acc:5487(6.231%) - Rem:0(0.000%) - Err:3438.503418
2022-04-23 00:31:12 - fullrmc <INFO> @0 Gen:88277 - Tr:25954(29.401%) - Acc:5488(6.217%) - Rem:0(0.000%) - Err:3438.310303
2022-04-23 00:31:12 - fullrmc <INFO> @0 Gen:88347 - Tr:25976(29.402%) - Acc:5489(6.213%) - Rem:0(0.000%) - Err:3438.175781
2022-04-23 00:31:12 - fullrmc <INFO> @0 Gen:88474 - Tr:26014(29.403%) - Acc:5490(6.205%) - Rem:0(0.000%) - Err:3438.130615
2022-04-23 00:31:13 - fullrmc <INFO> @0 Gen:88503 - Tr:26022(29.402%) - Acc:5491(6.204%) - Rem:0(0.000%) - Err:3437.256348
2022-04-23 00:31:13 - fullrmc <INFO> @0 Gen:88582 - Tr:26054(29.412%) - Acc:5492(6.200%) - Rem:0(0.000%) - Err:3437.1723

2022-04-23 00:31:36 - fullrmc <INFO> @0 Gen:92729 - Tr:27275(29.414%) - Acc:5547(5.982%) - Rem:0(0.000%) - Err:3411.004883
2022-04-23 00:31:36 - fullrmc <INFO> @0 Gen:92731 - Tr:27276(29.414%) - Acc:5548(5.983%) - Rem:0(0.000%) - Err:3410.672852
2022-04-23 00:31:36 - fullrmc <INFO> @0 Gen:92803 - Tr:27304(29.421%) - Acc:5549(5.979%) - Rem:0(0.000%) - Err:3410.266113
2022-04-23 00:31:36 - fullrmc <INFO> @0 Gen:92916 - Tr:27343(29.428%) - Acc:5550(5.973%) - Rem:0(0.000%) - Err:3410.120117
2022-04-23 00:31:36 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 00:31:40 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 00:31:40 - fullrmc <INFO> @0 Gen:93048 - Tr:27377(29.422%) - Acc:5551(5.966%) - Rem:0(0.000%) - Err:3407.920654
2022-04-23 00:31:40 - fullrmc <INFO> @0 Gen:93056 - Tr:27380(29.423%) - Acc:5552(5.966%) - Rem:0(0.000%) - Err:3407.304199
2022-04-23 00:31:40 - fullrmc <INFO> @0 Gen:93134 - Tr:27404(29.424%) - Acc:5553(5.962%) - Rem:0(0.000%) - Err:3407.1623

2022-04-23 00:32:09 - fullrmc <INFO> @0 Gen:98211 - Tr:28877(29.403%) - Acc:5607(5.709%) - Rem:0(0.000%) - Err:3382.457520
2022-04-23 00:32:09 - fullrmc <INFO> @0 Gen:98218 - Tr:28878(29.402%) - Acc:5608(5.710%) - Rem:0(0.000%) - Err:3381.390869
2022-04-23 00:32:10 - fullrmc <INFO> @0 Gen:98261 - Tr:28891(29.402%) - Acc:5609(5.708%) - Rem:0(0.000%) - Err:3381.305176
2022-04-23 00:32:10 - fullrmc <INFO> @0 Gen:98285 - Tr:28899(29.403%) - Acc:5610(5.708%) - Rem:0(0.000%) - Err:3381.079102
2022-04-23 00:32:10 - fullrmc <INFO> @0 Gen:98352 - Tr:28921(29.406%) - Acc:5611(5.705%) - Rem:0(0.000%) - Err:3380.809326
2022-04-23 00:32:10 - fullrmc <INFO> @0 Gen:98365 - Tr:28925(29.406%) - Acc:5612(5.705%) - Rem:0(0.000%) - Err:3379.537842
2022-04-23 00:32:10 - fullrmc <INFO> @0 Gen:98367 - Tr:28927(29.407%) - Acc:5613(5.706%) - Rem:0(0.000%) - Err:3378.652832
2022-04-23 00:32:10 - fullrmc <INFO> @0 Gen:98382 - Tr:28931(29.407%) - Acc:5614(5.706%) - Rem:0(0.000%) - Err:3378.152832
2022-04-23 00:32

In [13]:
ENGINE.set_groups_as_atoms()
ENGINE.run(numberOfSteps=1000000, saveFrequency=10000)

2022-04-23 00:32:21 - fullrmc <INFO> Engine @0 started 1000000 steps, total standard error is: 3371.894043
2022-04-23 00:32:21 - fullrmc <INFO> @0 Gen:100043 - Tr:29399(29.386%) - Acc:5631(5.629%) - Rem:0(0.000%) - Err:3371.564941
2022-04-23 00:32:21 - fullrmc <INFO> @0 Gen:100076 - Tr:29410(29.388%) - Acc:5632(5.628%) - Rem:0(0.000%) - Err:3370.459717
2022-04-23 00:32:22 - fullrmc <INFO> @0 Gen:100375 - Tr:29491(29.381%) - Acc:5633(5.612%) - Rem:0(0.000%) - Err:3370.424316
2022-04-23 00:32:22 - fullrmc <INFO> @0 Gen:100574 - Tr:29545(29.376%) - Acc:5634(5.602%) - Rem:0(0.000%) - Err:3370.202637
2022-04-23 00:32:22 - fullrmc <INFO> @0 Gen:100612 - Tr:29554(29.374%) - Acc:5635(5.601%) - Rem:0(0.000%) - Err:3370.042236
2022-04-23 00:32:22 - fullrmc <INFO> @0 Gen:100651 - Tr:29560(29.369%) - Acc:5636(5.600%) - Rem:0(0.000%) - Err:3369.271240
2022-04-23 00:32:23 - fullrmc <INFO> @0 Gen:100917 - Tr:29635(29.366%) - Acc:5637(5.586%) - Rem:0(0.000%) - Err:3369.150391
2022-04-23 00:32:23 - ful

2022-04-23 00:32:33 - fullrmc <INFO> @0 Gen:106309 - Tr:31136(29.288%) - Acc:5697(5.359%) - Rem:0(0.000%) - Err:3347.517822
2022-04-23 00:32:33 - fullrmc <INFO> @0 Gen:106405 - Tr:31172(29.296%) - Acc:5698(5.355%) - Rem:0(0.000%) - Err:3347.176270
2022-04-23 00:32:33 - fullrmc <INFO> @0 Gen:106419 - Tr:31174(29.294%) - Acc:5699(5.355%) - Rem:0(0.000%) - Err:3346.594727
2022-04-23 00:32:33 - fullrmc <INFO> @0 Gen:106552 - Tr:31214(29.295%) - Acc:5700(5.350%) - Rem:0(0.000%) - Err:3346.276855
2022-04-23 00:32:33 - fullrmc <INFO> @0 Gen:106573 - Tr:31223(29.297%) - Acc:5701(5.349%) - Rem:0(0.000%) - Err:3345.496826
2022-04-23 00:32:33 - fullrmc <INFO> @0 Gen:106707 - Tr:31269(29.304%) - Acc:5702(5.344%) - Rem:0(0.000%) - Err:3344.315430
2022-04-23 00:32:34 - fullrmc <INFO> @0 Gen:106832 - Tr:31305(29.303%) - Acc:5703(5.338%) - Rem:0(0.000%) - Err:3343.660156
2022-04-23 00:32:34 - fullrmc <INFO> @0 Gen:106911 - Tr:31328(29.303%) - Acc:5704(5.335%) - Rem:0(0.000%) - Err:3343.181885
2022-04-

2022-04-23 00:32:47 - fullrmc <INFO> @0 Gen:111829 - Tr:32787(29.319%) - Acc:5762(5.153%) - Rem:0(0.000%) - Err:3320.876953
2022-04-23 00:32:47 - fullrmc <INFO> @0 Gen:111872 - Tr:32799(29.318%) - Acc:5763(5.151%) - Rem:0(0.000%) - Err:3320.651855
2022-04-23 00:32:48 - fullrmc <INFO> @0 Gen:112117 - Tr:32874(29.321%) - Acc:5764(5.141%) - Rem:0(0.000%) - Err:3319.248047
2022-04-23 00:32:49 - fullrmc <INFO> @0 Gen:112755 - Tr:33046(29.308%) - Acc:5765(5.113%) - Rem:0(0.000%) - Err:3318.729736
2022-04-23 00:32:49 - fullrmc <INFO> @0 Gen:112815 - Tr:33064(29.308%) - Acc:5766(5.111%) - Rem:0(0.000%) - Err:3318.074707
2022-04-23 00:32:49 - fullrmc <INFO> @0 Gen:112876 - Tr:33084(29.310%) - Acc:5767(5.109%) - Rem:0(0.000%) - Err:3317.929688
2022-04-23 00:32:49 - fullrmc <INFO> @0 Gen:113196 - Tr:33164(29.298%) - Acc:5768(5.096%) - Rem:0(0.000%) - Err:3317.656738
2022-04-23 00:32:50 - fullrmc <INFO> @0 Gen:113310 - Tr:33198(29.298%) - Acc:5769(5.091%) - Rem:0(0.000%) - Err:3317.468262
2022-04-

2022-04-23 00:33:09 - fullrmc <INFO> @0 Gen:121777 - Tr:35667(29.289%) - Acc:5827(4.785%) - Rem:0(0.000%) - Err:3300.148438
2022-04-23 00:33:09 - fullrmc <INFO> @0 Gen:121846 - Tr:35679(29.282%) - Acc:5828(4.783%) - Rem:0(0.000%) - Err:3300.099854
2022-04-23 00:33:09 - fullrmc <INFO> @0 Gen:122030 - Tr:35747(29.294%) - Acc:5829(4.777%) - Rem:0(0.000%) - Err:3299.849365
2022-04-23 00:33:10 - fullrmc <INFO> @0 Gen:122253 - Tr:35819(29.299%) - Acc:5830(4.769%) - Rem:0(0.000%) - Err:3299.448486
2022-04-23 00:33:10 - fullrmc <INFO> @0 Gen:122598 - Tr:35910(29.291%) - Acc:5831(4.756%) - Rem:0(0.000%) - Err:3299.390137
2022-04-23 00:33:10 - fullrmc <INFO> @0 Gen:122633 - Tr:35918(29.289%) - Acc:5832(4.756%) - Rem:0(0.000%) - Err:3298.971436
2022-04-23 00:33:10 - fullrmc <INFO> @0 Gen:122663 - Tr:35926(29.288%) - Acc:5833(4.755%) - Rem:0(0.000%) - Err:3298.933350
2022-04-23 00:33:11 - fullrmc <INFO> @0 Gen:122734 - Tr:35947(29.289%) - Acc:5834(4.753%) - Rem:0(0.000%) - Err:3298.813477
2022-04-

2022-04-23 00:33:21 - fullrmc <INFO> @0 Gen:128541 - Tr:37648(29.289%) - Acc:5894(4.585%) - Rem:0(0.000%) - Err:3269.978271
2022-04-23 00:33:22 - fullrmc <INFO> @0 Gen:128660 - Tr:37684(29.290%) - Acc:5895(4.582%) - Rem:0(0.000%) - Err:3269.772705
2022-04-23 00:33:22 - fullrmc <INFO> @0 Gen:128742 - Tr:37712(29.293%) - Acc:5896(4.580%) - Rem:0(0.000%) - Err:3269.731934
2022-04-23 00:33:22 - fullrmc <INFO> @0 Gen:128821 - Tr:37735(29.293%) - Acc:5897(4.578%) - Rem:0(0.000%) - Err:3269.685059
2022-04-23 00:33:22 - fullrmc <INFO> @0 Gen:128838 - Tr:37743(29.295%) - Acc:5898(4.578%) - Rem:0(0.000%) - Err:3269.626221
2022-04-23 00:33:22 - fullrmc <INFO> @0 Gen:128929 - Tr:37771(29.296%) - Acc:5899(4.575%) - Rem:0(0.000%) - Err:3268.774658
2022-04-23 00:33:22 - fullrmc <INFO> @0 Gen:129014 - Tr:37797(29.297%) - Acc:5900(4.573%) - Rem:0(0.000%) - Err:3268.216553
2022-04-23 00:33:22 - fullrmc <INFO> @0 Gen:129063 - Tr:37813(29.298%) - Acc:5901(4.572%) - Rem:0(0.000%) - Err:3267.980713
2022-04-

...

2022-04-23 01:01:59 - fullrmc <INFO> @0 Gen:942839 - Tr:275724(29.244%) - Acc:8779(0.931%) - Rem:0(0.000%) - Err:2704.289307
2022-04-23 01:02:00 - fullrmc <INFO> @0 Gen:943019 - Tr:275782(29.245%) - Acc:8780(0.931%) - Rem:0(0.000%) - Err:2703.849609
2022-04-23 01:02:00 - fullrmc <INFO> @0 Gen:943051 - Tr:275796(29.245%) - Acc:8781(0.931%) - Rem:0(0.000%) - Err:2703.758301
2022-04-23 01:02:01 - fullrmc <INFO> @0 Gen:943429 - Tr:275895(29.244%) - Acc:8782(0.931%) - Rem:0(0.000%) - Err:2703.735352
2022-04-23 01:02:01 - fullrmc <INFO> @0 Gen:943988 - Tr:276061(29.244%) - Acc:8783(0.930%) - Rem:0(0.000%) - Err:2703.642090
2022-04-23 01:02:03 - fullrmc <INFO> @0 Gen:944980 - Tr:276330(29.242%) - Acc:8784(0.930%) - Rem:0(0.000%) - Err:2703.631348
2022-04-23 01:02:04 - fullrmc <INFO> @0 Gen:945668 - Tr:276517(29.240%) - Acc:8785(0.929%) - Rem:0(0.000%) - Err:2703.626221
2022-04-23 01:02:05 - fullrmc <INFO> @0 Gen:946108 - Tr:276652(29.241%) - Acc:8786(0.929%) - Rem:0(0.000%) - Err:2703.552979


2022-04-23 01:03:05 - fullrmc <INFO> @0 Gen:975598 - Tr:285361(29.250%) - Acc:8842(0.906%) - Rem:0(0.000%) - Err:2698.801025
2022-04-23 01:03:06 - fullrmc <INFO> @0 Gen:975866 - Tr:285438(29.250%) - Acc:8843(0.906%) - Rem:0(0.000%) - Err:2698.763428
2022-04-23 01:03:06 - fullrmc <INFO> @0 Gen:975887 - Tr:285445(29.250%) - Acc:8844(0.906%) - Rem:0(0.000%) - Err:2698.732910
2022-04-23 01:03:06 - fullrmc <INFO> @0 Gen:976183 - Tr:285516(29.248%) - Acc:8845(0.906%) - Rem:0(0.000%) - Err:2698.665527
2022-04-23 01:03:06 - fullrmc <INFO> @0 Gen:976239 - Tr:285531(29.248%) - Acc:8846(0.906%) - Rem:0(0.000%) - Err:2698.642822
2022-04-23 01:03:07 - fullrmc <INFO> @0 Gen:976659 - Tr:285663(29.249%) - Acc:8847(0.906%) - Rem:0(0.000%) - Err:2698.599854
2022-04-23 01:03:07 - fullrmc <INFO> @0 Gen:976715 - Tr:285686(29.250%) - Acc:8848(0.906%) - Rem:0(0.000%) - Err:2698.599854
2022-04-23 01:03:07 - fullrmc <INFO> @0 Gen:976878 - Tr:285734(29.250%) - Acc:8849(0.906%) - Rem:0(0.000%) - Err:2698.473389


2022-04-23 01:04:01 - fullrmc <INFO> @0 Gen:1001633 - Tr:293145(29.267%) - Acc:8905(0.889%) - Rem:0(0.000%) - Err:2693.022705
2022-04-23 01:04:03 - fullrmc <INFO> @0 Gen:1003052 - Tr:293571(29.268%) - Acc:8906(0.888%) - Rem:0(0.000%) - Err:2693.012207
2022-04-23 01:04:03 - fullrmc <INFO> @0 Gen:1003186 - Tr:293619(29.269%) - Acc:8907(0.888%) - Rem:0(0.000%) - Err:2692.919678
2022-04-23 01:04:06 - fullrmc <INFO> @0 Gen:1004853 - Tr:294104(29.268%) - Acc:8908(0.886%) - Rem:0(0.000%) - Err:2692.775879
2022-04-23 01:04:06 - fullrmc <INFO> @0 Gen:1004924 - Tr:294127(29.269%) - Acc:8909(0.887%) - Rem:0(0.000%) - Err:2692.768799
2022-04-23 01:04:07 - fullrmc <INFO> @0 Gen:1005774 - Tr:294379(29.269%) - Acc:8910(0.886%) - Rem:0(0.000%) - Err:2692.761230
2022-04-23 01:04:08 - fullrmc <INFO> @0 Gen:1006387 - Tr:294554(29.268%) - Acc:8911(0.885%) - Rem:0(0.000%) - Err:2692.638184
2022-04-23 01:04:09 - fullrmc <INFO> @0 Gen:1006571 - Tr:294614(29.269%) - Acc:8912(0.885%) - Rem:0(0.000%) - Err:2692

2022-04-23 01:05:24 - fullrmc <INFO> @0 Gen:1043238 - Tr:305425(29.277%) - Acc:8966(0.859%) - Rem:0(0.000%) - Err:2687.162354
2022-04-23 01:05:25 - fullrmc <INFO> @0 Gen:1044146 - Tr:305670(29.275%) - Acc:8967(0.859%) - Rem:0(0.000%) - Err:2685.772217
2022-04-23 01:05:28 - fullrmc <INFO> @0 Gen:1045740 - Tr:306099(29.271%) - Acc:8968(0.858%) - Rem:0(0.000%) - Err:2685.772217
2022-04-23 01:05:29 - fullrmc <INFO> @0 Gen:1046332 - Tr:306264(29.270%) - Acc:8969(0.857%) - Rem:0(0.000%) - Err:2685.713135
2022-04-23 01:05:31 - fullrmc <INFO> @0 Gen:1047829 - Tr:306731(29.273%) - Acc:8970(0.856%) - Rem:0(0.000%) - Err:2685.653809
2022-04-23 01:05:31 - fullrmc <INFO> @0 Gen:1047969 - Tr:306770(29.273%) - Acc:8971(0.856%) - Rem:0(0.000%) - Err:2685.653809
2022-04-23 01:05:32 - fullrmc <INFO> @0 Gen:1048111 - Tr:306820(29.274%) - Acc:8972(0.856%) - Rem:0(0.000%) - Err:2685.637451
2022-04-23 01:05:32 - fullrmc <INFO> @0 Gen:1048128 - Tr:306825(29.274%) - Acc:8973(0.856%) - Rem:0(0.000%) - Err:2685

2022-04-23 01:06:32 - fullrmc <INFO> @0 Gen:1077546 - Tr:315468(29.277%) - Acc:9028(0.838%) - Rem:0(0.000%) - Err:2680.768555
2022-04-23 01:06:33 - fullrmc <INFO> @0 Gen:1078534 - Tr:315751(29.276%) - Acc:9029(0.837%) - Rem:0(0.000%) - Err:2680.708008
2022-04-23 01:06:34 - fullrmc <INFO> @0 Gen:1078747 - Tr:315816(29.276%) - Acc:9030(0.837%) - Rem:0(0.000%) - Err:2680.565918
2022-04-23 01:06:36 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 01:06:40 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 01:06:40 - fullrmc <INFO> @0 Gen:1080284 - Tr:316253(29.275%) - Acc:9031(0.836%) - Rem:0(0.000%) - Err:2680.464844
2022-04-23 01:06:41 - fullrmc <INFO> @0 Gen:1080648 - Tr:316366(29.276%) - Acc:9032(0.836%) - Rem:0(0.000%) - Err:2680.461182
2022-04-23 01:06:45 - fullrmc <INFO> @0 Gen:1083420 - Tr:317168(29.275%) - Acc:9033(0.834%) - Rem:0(0.000%) - Err:2680.435303
2022-04-23 01:06:45 - fullrmc <INFO> @0 Gen:1083516 - Tr:317199(29.275%) - Acc:9034(0.834%) - Rem:0(0.

In [14]:
ENGINE.set_groups_as_atoms()
ENGINE.run(numberOfSteps=100000, saveFrequency=1000)

2022-04-23 01:07:21 - fullrmc <INFO> Engine @0 started 100000 steps, total standard error is: 2678.718750
2022-04-23 01:07:22 - fullrmc <INFO> @0 Gen:1100719 - Tr:322293(29.280%) - Acc:9061(0.823%) - Rem:0(0.000%) - Err:2678.617188
2022-04-23 01:07:22 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 01:07:26 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 01:07:26 - fullrmc <INFO> @0 Gen:1101013 - Tr:322383(29.281%) - Acc:9062(0.823%) - Rem:0(0.000%) - Err:2678.605469
2022-04-23 01:07:27 - fullrmc <INFO> @0 Gen:1101199 - Tr:322449(29.282%) - Acc:9063(0.823%) - Rem:0(0.000%) - Err:2678.541992
2022-04-23 01:07:27 - fullrmc <INFO> @0 Gen:1101206 - Tr:322452(29.282%) - Acc:9064(0.823%) - Rem:0(0.000%) - Err:2678.503662
2022-04-23 01:07:27 - fullrmc <INFO> @0 Gen:1101243 - Tr:322469(29.282%) - Acc:9065(0.823%) - Rem:0(0.000%) - Err:2678.423828
2022-04-23 01:07:27 - fullrmc <INFO> @0 Gen:1101282 - Tr:322479(29.282%) - Acc:9066(0.823%) - Rem:0(0.000%) - Err:2678.401

2022-04-23 01:08:48 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 01:08:52 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 01:08:53 - fullrmc <INFO> @0 Gen:1117791 - Tr:327328(29.283%) - Acc:9108(0.815%) - Rem:0(0.000%) - Err:2672.528320
2022-04-23 01:08:53 - fullrmc <INFO> @0 Gen:1117979 - Tr:327386(29.284%) - Acc:9109(0.815%) - Rem:0(0.000%) - Err:2672.295898
2022-04-23 01:08:54 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 01:08:57 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 01:08:57 - fullrmc <INFO> @0 Gen:1118033 - Tr:327402(29.284%) - Acc:9110(0.815%) - Rem:0(0.000%) - Err:2672.292969
2022-04-23 01:08:58 - fullrmc <INFO> @0 Gen:1118171 - Tr:327447(29.284%) - Acc:9111(0.815%) - Rem:0(0.000%) - Err:2672.279297
2022-04-23 01:08:59 - fullrmc <INFO> @0 Gen:1118803 - Tr:327632(29.284%) - Acc:9112(0.814%) - Rem:0(0.000%) - Err:2672.272949
2022-04-23 01:08:59 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 

2022-04-23 01:10:30 - fullrmc <INFO> @0 Gen:1135191 - Tr:332403(29.282%) - Acc:9152(0.806%) - Rem:0(0.000%) - Err:2669.210938
2022-04-23 01:10:30 - fullrmc <INFO> @0 Gen:1135372 - Tr:332455(29.282%) - Acc:9153(0.806%) - Rem:0(0.000%) - Err:2669.175049
2022-04-23 01:10:31 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 01:10:35 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 01:10:35 - fullrmc <INFO> @0 Gen:1136026 - Tr:332639(29.281%) - Acc:9154(0.806%) - Rem:0(0.000%) - Err:2669.139648
2022-04-23 01:10:36 - fullrmc <INFO> @0 Gen:1136369 - Tr:332742(29.281%) - Acc:9155(0.806%) - Rem:0(0.000%) - Err:2668.982910
2022-04-23 01:10:36 - fullrmc <INFO> @0 Gen:1136591 - Tr:332807(29.281%) - Acc:9156(0.806%) - Rem:0(0.000%) - Err:2668.875977
2022-04-23 01:10:37 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 01:10:41 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 01:10:41 - fullrmc <INFO> @0 Gen:1137266 - Tr:333018(29.282%) - Acc:9157(0.

2022-04-23 01:12:29 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 01:12:30 - fullrmc <INFO> @0 Gen:1160480 - Tr:339769(29.278%) - Acc:9191(0.792%) - Rem:0(0.000%) - Err:2665.847656
2022-04-23 01:12:30 - fullrmc <INFO> @0 Gen:1160840 - Tr:339876(29.278%) - Acc:9192(0.792%) - Rem:0(0.000%) - Err:2665.779053
2022-04-23 01:12:31 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 01:12:35 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 01:12:36 - fullrmc <INFO> @0 Gen:1161536 - Tr:340076(29.278%) - Acc:9193(0.791%) - Rem:0(0.000%) - Err:2665.766113
2022-04-23 01:12:36 - fullrmc <INFO> @0 Gen:1161543 - Tr:340077(29.278%) - Acc:9194(0.792%) - Rem:0(0.000%) - Err:2665.726318
2022-04-23 01:12:36 - fullrmc <INFO> @0 Gen:1161554 - Tr:340080(29.278%) - Acc:9195(0.792%) - Rem:0(0.000%) - Err:2665.501221
2022-04-23 01:12:36 - fullrmc <INFO> @0 Gen:1161595 - Tr:340093(29.278%) - Acc:9196(0.792%) - Rem:0(0.000%) - Err:2665.323242
2022-04-23 01:12:36 - fullrmc <INFO

2022-04-23 01:14:16 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 01:14:20 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 01:14:21 - fullrmc <INFO> @0 Gen:1182381 - Tr:346192(29.279%) - Acc:9233(0.781%) - Rem:0(0.000%) - Err:2661.507080
2022-04-23 01:14:22 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 01:14:26 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 01:14:27 - fullrmc <INFO> @0 Save engine omitted because no improvement made since last save.
2022-04-23 01:14:28 - fullrmc <INFO> @0 Gen:1184602 - Tr:346842(29.279%) - Acc:9234(0.780%) - Rem:0(0.000%) - Err:2661.480713
2022-04-23 01:14:28 - fullrmc <INFO> @0 Gen:1184798 - Tr:346893(29.279%) - Acc:9235(0.779%) - Rem:0(0.000%) - Err:2661.448975
2022-04-23 01:14:29 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 01:14:33 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 01:14:33 - fullrmc <INFO> @0 Gen:1185005 - Tr:346950(29.278%) - Acc:9236(0.779%)

In [15]:
ENGINE.set_groups_as_atoms()
ENGINE.run(numberOfSteps=1000000, saveFrequency=10000)

2022-04-23 01:15:50 - fullrmc <INFO> Engine @0 started 1000000 steps, total standard error is: 2656.924072
2022-04-23 01:15:51 - fullrmc <INFO> @0 Gen:1200085 - Tr:351310(29.274%) - Acc:9270(0.772%) - Rem:0(0.000%) - Err:2656.813477
2022-04-23 01:15:51 - fullrmc <INFO> @0 Gen:1200154 - Tr:351332(29.274%) - Acc:9271(0.772%) - Rem:0(0.000%) - Err:2656.813477
2022-04-23 01:15:51 - fullrmc <INFO> @0 Gen:1200307 - Tr:351387(29.275%) - Acc:9272(0.772%) - Rem:0(0.000%) - Err:2656.796875
2022-04-23 01:15:51 - fullrmc <INFO> @0 Gen:1200454 - Tr:351433(29.275%) - Acc:9273(0.772%) - Rem:0(0.000%) - Err:2656.791260
2022-04-23 01:15:52 - fullrmc <INFO> @0 Gen:1200903 - Tr:351561(29.275%) - Acc:9274(0.772%) - Rem:0(0.000%) - Err:2656.778809
2022-04-23 01:15:53 - fullrmc <INFO> @0 Gen:1201317 - Tr:351695(29.276%) - Acc:9275(0.772%) - Rem:0(0.000%) - Err:2656.778809
2022-04-23 01:15:53 - fullrmc <INFO> @0 Gen:1201548 - Tr:351761(29.276%) - Acc:9276(0.772%) - Rem:0(0.000%) - Err:2656.778809
2022-04-23 

2022-04-23 01:16:46 - fullrmc <INFO> @0 Gen:1228734 - Tr:359841(29.286%) - Acc:9332(0.759%) - Rem:0(0.000%) - Err:2653.647217
2022-04-23 01:16:46 - fullrmc <INFO> @0 Gen:1228809 - Tr:359863(29.286%) - Acc:9333(0.760%) - Rem:0(0.000%) - Err:2653.433594
2022-04-23 01:16:47 - fullrmc <INFO> @0 Gen:1228896 - Tr:359885(29.285%) - Acc:9334(0.760%) - Rem:0(0.000%) - Err:2653.360596
2022-04-23 01:16:47 - fullrmc <INFO> @0 Gen:1229196 - Tr:359978(29.286%) - Acc:9335(0.759%) - Rem:0(0.000%) - Err:2653.262695
2022-04-23 01:16:48 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 01:16:52 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 01:16:52 - fullrmc <INFO> @0 Gen:1230204 - Tr:360255(29.284%) - Acc:9336(0.759%) - Rem:0(0.000%) - Err:2653.195557
2022-04-23 01:16:53 - fullrmc <INFO> @0 Gen:1230628 - Tr:360370(29.283%) - Acc:9337(0.759%) - Rem:0(0.000%) - Err:2652.992676
2022-04-23 01:16:54 - fullrmc <INFO> @0 Gen:1231049 - Tr:360500(29.284%) - Acc:9338(0.759%) - Rem:0(0.

2022-04-23 01:17:54 - fullrmc <INFO> @0 Gen:1260447 - Tr:369159(29.288%) - Acc:9393(0.745%) - Rem:0(0.000%) - Err:2647.632080
2022-04-23 01:17:54 - fullrmc <INFO> @0 Gen:1260671 - Tr:369226(29.288%) - Acc:9394(0.745%) - Rem:0(0.000%) - Err:2647.584961
2022-04-23 01:17:55 - fullrmc <INFO> @0 Gen:1261010 - Tr:369328(29.288%) - Acc:9395(0.745%) - Rem:0(0.000%) - Err:2647.427734
2022-04-23 01:17:57 - fullrmc <INFO> @0 Gen:1262319 - Tr:369743(29.291%) - Acc:9396(0.744%) - Rem:0(0.000%) - Err:2647.417725
2022-04-23 01:17:58 - fullrmc <INFO> @0 Gen:1262781 - Tr:369864(29.290%) - Acc:9397(0.744%) - Rem:0(0.000%) - Err:2647.417725
2022-04-23 01:18:00 - fullrmc <INFO> @0 Gen:1264058 - Tr:370233(29.289%) - Acc:9398(0.743%) - Rem:0(0.000%) - Err:2647.324219
2022-04-23 01:18:00 - fullrmc <INFO> @0 Gen:1264208 - Tr:370276(29.289%) - Acc:9399(0.743%) - Rem:0(0.000%) - Err:2647.232910
2022-04-23 01:18:01 - fullrmc <INFO> @0 Gen:1264797 - Tr:370462(29.290%) - Acc:9400(0.743%) - Rem:0(0.000%) - Err:2647

2022-04-23 01:19:17 - fullrmc <INFO> @0 Gen:1301430 - Tr:381174(29.289%) - Acc:9454(0.726%) - Rem:0(0.000%) - Err:2642.975098
2022-04-23 01:19:17 - fullrmc <INFO> @0 Gen:1301810 - Tr:381279(29.288%) - Acc:9455(0.726%) - Rem:0(0.000%) - Err:2642.960938
2022-04-23 01:19:18 - fullrmc <INFO> @0 Gen:1301982 - Tr:381322(29.288%) - Acc:9456(0.726%) - Rem:0(0.000%) - Err:2642.946777
2022-04-23 01:19:18 - fullrmc <INFO> @0 Gen:1302347 - Tr:381431(29.288%) - Acc:9457(0.726%) - Rem:0(0.000%) - Err:2642.946289
2022-04-23 01:19:18 - fullrmc <INFO> @0 Gen:1302437 - Tr:381463(29.288%) - Acc:9458(0.726%) - Rem:0(0.000%) - Err:2642.879395
2022-04-23 01:19:21 - fullrmc <INFO> @0 Gen:1303890 - Tr:381864(29.287%) - Acc:9459(0.725%) - Rem:0(0.000%) - Err:2642.875000
2022-04-23 01:19:23 - fullrmc <INFO> @0 Gen:1305375 - Tr:382303(29.287%) - Acc:9460(0.725%) - Rem:0(0.000%) - Err:2642.808838
2022-04-23 01:19:29 - fullrmc <INFO> @0 Gen:1308933 - Tr:383374(29.289%) - Acc:9461(0.723%) - Rem:0(0.000%) - Err:2642

2022-04-23 01:20:32 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 01:20:36 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 01:20:37 - fullrmc <INFO> @0 Gen:1340682 - Tr:392691(29.290%) - Acc:9516(0.710%) - Rem:0(0.000%) - Err:2638.144531
2022-04-23 01:20:38 - fullrmc <INFO> @0 Gen:1341039 - Tr:392804(29.291%) - Acc:9517(0.710%) - Rem:0(0.000%) - Err:2638.141113
2022-04-23 01:20:38 - fullrmc <INFO> @0 Gen:1341161 - Tr:392835(29.291%) - Acc:9518(0.710%) - Rem:0(0.000%) - Err:2637.938477
2022-04-23 01:20:39 - fullrmc <INFO> @0 Gen:1341499 - Tr:392924(29.290%) - Acc:9519(0.710%) - Rem:0(0.000%) - Err:2637.424805
2022-04-23 01:20:39 - fullrmc <INFO> @0 Gen:1341974 - Tr:393053(29.289%) - Acc:9520(0.709%) - Rem:0(0.000%) - Err:2637.364258
2022-04-23 01:20:40 - fullrmc <INFO> @0 Gen:1342126 - Tr:393099(29.289%) - Acc:9521(0.709%) - Rem:0(0.000%) - Err:2637.362549
2022-04-23 01:20:40 - fullrmc <INFO> @0 Gen:1342243 - Tr:393134(29.289%) - Acc:9522(0.709%) - Rem:0(0.

...

2022-04-23 01:41:47 - fullrmc <INFO> @0 Gen:1967034 - Tr:578231(29.396%) - Acc:10492(0.533%) - Rem:0(0.000%) - Err:2567.187256
2022-04-23 01:41:50 - fullrmc <INFO> @0 Gen:1969232 - Tr:578861(29.395%) - Acc:10493(0.533%) - Rem:0(0.000%) - Err:2567.120361
2022-04-23 01:41:51 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 01:41:55 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 01:41:57 - fullrmc <INFO> @0 Gen:1971425 - Tr:579508(29.395%) - Acc:10494(0.532%) - Rem:0(0.000%) - Err:2566.909912
2022-04-23 01:41:58 - fullrmc <INFO> @0 Gen:1971663 - Tr:579584(29.396%) - Acc:10495(0.532%) - Rem:0(0.000%) - Err:2566.835938
2022-04-23 01:42:00 - fullrmc <INFO> @0 Gen:1972793 - Tr:579915(29.396%) - Acc:10496(0.532%) - Rem:0(0.000%) - Err:2566.734375
2022-04-23 01:42:01 - fullrmc <INFO> @0 Gen:1973849 - Tr:580199(29.394%) - Acc:10497(0.532%) - Rem:0(0.000%) - Err:2566.698242
2022-04-23 01:42:03 - fullrmc <INFO> @0 Gen:1974691 - Tr:580463(29.395%) - Acc:10498(0.532%) - R

2022-04-23 01:43:29 - fullrmc <INFO> @0 Gen:2018185 - Tr:593289(29.397%) - Acc:10551(0.523%) - Rem:0(0.000%) - Err:2563.759033
2022-04-23 01:43:31 - fullrmc <INFO> @0 Gen:2019126 - Tr:593599(29.399%) - Acc:10552(0.523%) - Rem:0(0.000%) - Err:2563.748535
2022-04-23 01:43:32 - fullrmc <INFO> @0 Gen:2019968 - Tr:593852(29.399%) - Acc:10553(0.522%) - Rem:0(0.000%) - Err:2563.647949
2022-04-23 01:43:32 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 01:43:36 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 01:43:37 - fullrmc <INFO> @0 Gen:2020850 - Tr:594119(29.399%) - Acc:10554(0.522%) - Rem:0(0.000%) - Err:2563.456055
2022-04-23 01:43:38 - fullrmc <INFO> @0 Gen:2021105 - Tr:594188(29.399%) - Acc:10555(0.522%) - Rem:0(0.000%) - Err:2563.436523
2022-04-23 01:43:38 - fullrmc <INFO> @0 Gen:2021180 - Tr:594212(29.399%) - Acc:10556(0.522%) - Rem:0(0.000%) - Err:2563.402344
2022-04-23 01:43:39 - fullrmc <INFO> @0 Gen:2021973 - Tr:594431(29.399%) - Acc:10557(0.522%) - R

2022-04-23 01:44:58 - fullrmc <INFO> @0 Gen:2060646 - Tr:605843(29.401%) - Acc:10610(0.515%) - Rem:0(0.000%) - Err:2559.990723
2022-04-23 01:44:59 - fullrmc <INFO> @0 Gen:2061410 - Tr:606055(29.400%) - Acc:10611(0.515%) - Rem:0(0.000%) - Err:2559.736816
2022-04-23 01:45:01 - fullrmc <INFO> @0 Gen:2062188 - Tr:606286(29.400%) - Acc:10612(0.515%) - Rem:0(0.000%) - Err:2559.680664
2022-04-23 01:45:01 - fullrmc <INFO> @0 Gen:2062551 - Tr:606384(29.400%) - Acc:10613(0.515%) - Rem:0(0.000%) - Err:2559.594971
2022-04-23 01:45:03 - fullrmc <INFO> @0 Gen:2063326 - Tr:606618(29.400%) - Acc:10614(0.514%) - Rem:0(0.000%) - Err:2559.469971
2022-04-23 01:45:04 - fullrmc <INFO> @0 Gen:2063960 - Tr:606808(29.400%) - Acc:10615(0.514%) - Rem:0(0.000%) - Err:2559.464844
2022-04-23 01:45:04 - fullrmc <INFO> @0 Gen:2064095 - Tr:606851(29.400%) - Acc:10616(0.514%) - Rem:0(0.000%) - Err:2559.457275
2022-04-23 01:45:05 - fullrmc <INFO> @0 Gen:2065017 - Tr:607126(29.401%) - Acc:10617(0.514%) - Rem:0(0.000%) - 

2022-04-23 01:46:25 - fullrmc <INFO> @0 Gen:2105012 - Tr:618946(29.403%) - Acc:10671(0.507%) - Rem:0(0.000%) - Err:2556.194336
2022-04-23 01:46:29 - fullrmc <INFO> @0 Gen:2107088 - Tr:619575(29.404%) - Acc:10672(0.506%) - Rem:0(0.000%) - Err:2556.175537
2022-04-23 01:46:32 - fullrmc <INFO> @0 Gen:2109287 - Tr:620243(29.405%) - Acc:10673(0.506%) - Rem:0(0.000%) - Err:2556.166748
2022-04-23 01:46:33 - fullrmc <INFO> @0 Gen:2109841 - Tr:620398(29.405%) - Acc:10674(0.506%) - Rem:0(0.000%) - Err:2556.135498
2022-04-23 01:46:33 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 01:46:37 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 01:46:40 - fullrmc <INFO> @0 Gen:2111493 - Tr:620849(29.403%) - Acc:10675(0.506%) - Rem:0(0.000%) - Err:2556.121338
2022-04-23 01:46:43 - fullrmc <INFO> @0 Gen:2113375 - Tr:621404(29.403%) - Acc:10676(0.505%) - Rem:0(0.000%) - Err:2555.979248
2022-04-23 01:46:44 - fullrmc <INFO> @0 Gen:2114137 - Tr:621645(29.404%) - Acc:10677(0.505%) - R

2022-04-23 01:48:06 - fullrmc <INFO> @0 Gen:2155187 - Tr:633834(29.410%) - Acc:10730(0.498%) - Rem:0(0.000%) - Err:2552.137207
2022-04-23 01:48:09 - fullrmc <INFO> @0 Gen:2157053 - Tr:634393(29.410%) - Acc:10731(0.497%) - Rem:0(0.000%) - Err:2552.115234
2022-04-23 01:48:09 - fullrmc <INFO> @0 Gen:2157148 - Tr:634421(29.410%) - Acc:10732(0.498%) - Rem:0(0.000%) - Err:2552.095215
2022-04-23 01:48:11 - fullrmc <INFO> @0 Gen:2157809 - Tr:634621(29.410%) - Acc:10733(0.497%) - Rem:0(0.000%) - Err:2552.035156
2022-04-23 01:48:12 - fullrmc <INFO> @0 Gen:2158929 - Tr:634931(29.410%) - Acc:10734(0.497%) - Rem:0(0.000%) - Err:2551.965332
2022-04-23 01:48:13 - fullrmc <INFO> @0 Gen:2159066 - Tr:634969(29.409%) - Acc:10735(0.497%) - Rem:0(0.000%) - Err:2551.961426
2022-04-23 01:48:14 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 01:48:18 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 01:48:19 - fullrmc <INFO> @0 Gen:2160507 - Tr:635395(29.410%) - Acc:10736(0.497%) - R

In [16]:
ENGINE.set_groups_as_atoms()
ENGINE.run(numberOfSteps=5000000, saveFrequency=50000)

2022-04-23 01:49:40 - fullrmc <INFO> Engine @0 started 5000000 steps, total standard error is: 2548.549316
2022-04-23 01:49:42 - fullrmc <INFO> @0 Gen:2201379 - Tr:647424(29.410%) - Acc:10788(0.490%) - Rem:0(0.000%) - Err:2548.524902
2022-04-23 01:49:42 - fullrmc <INFO> @0 Gen:2201521 - Tr:647470(29.410%) - Acc:10789(0.490%) - Rem:0(0.000%) - Err:2548.472656
2022-04-23 01:49:43 - fullrmc <INFO> @0 Gen:2202316 - Tr:647694(29.410%) - Acc:10790(0.490%) - Rem:0(0.000%) - Err:2548.339844
2022-04-23 01:49:45 - fullrmc <INFO> @0 Gen:2203163 - Tr:647952(29.410%) - Acc:10791(0.490%) - Rem:0(0.000%) - Err:2548.339844
2022-04-23 01:49:45 - fullrmc <INFO> @0 Gen:2203232 - Tr:647978(29.410%) - Acc:10792(0.490%) - Rem:0(0.000%) - Err:2548.298828
2022-04-23 01:49:48 - fullrmc <INFO> @0 Gen:2204851 - Tr:648488(29.412%) - Acc:10793(0.490%) - Rem:0(0.000%) - Err:2548.290283
2022-04-23 01:49:49 - fullrmc <INFO> @0 Gen:2205525 - Tr:648701(29.413%) - Acc:10794(0.489%) - Rem:0(0.000%) - Err:2548.280273
2022

2022-04-23 01:51:08 - fullrmc <INFO> @0 Gen:2251722 - Tr:662382(29.417%) - Acc:10851(0.482%) - Rem:0(0.000%) - Err:2545.036133
2022-04-23 01:51:09 - fullrmc <INFO> @0 Gen:2251982 - Tr:662461(29.417%) - Acc:10852(0.482%) - Rem:0(0.000%) - Err:2545.036133
2022-04-23 01:51:09 - fullrmc <INFO> @0 Gen:2252046 - Tr:662485(29.417%) - Acc:10853(0.482%) - Rem:0(0.000%) - Err:2545.031494
2022-04-23 01:51:12 - fullrmc <INFO> @0 Gen:2254006 - Tr:663079(29.418%) - Acc:10854(0.482%) - Rem:0(0.000%) - Err:2544.970703
2022-04-23 01:51:12 - fullrmc <INFO> @0 Gen:2254053 - Tr:663091(29.418%) - Acc:10855(0.482%) - Rem:0(0.000%) - Err:2544.913818
2022-04-23 01:51:12 - fullrmc <INFO> @0 Gen:2254376 - Tr:663181(29.417%) - Acc:10856(0.482%) - Rem:0(0.000%) - Err:2544.910156
2022-04-23 01:51:16 - fullrmc <INFO> @0 Gen:2256692 - Tr:663878(29.418%) - Acc:10857(0.481%) - Rem:0(0.000%) - Err:2544.843262
2022-04-23 01:51:18 - fullrmc <INFO> @0 Gen:2257876 - Tr:664233(29.418%) - Acc:10858(0.481%) - Rem:0(0.000%) - 

2022-04-23 01:52:42 - fullrmc <INFO> @0 Gen:2306788 - Tr:678802(29.426%) - Acc:10915(0.473%) - Rem:0(0.000%) - Err:2542.024414
2022-04-23 01:52:42 - fullrmc <INFO> @0 Gen:2307172 - Tr:678909(29.426%) - Acc:10916(0.473%) - Rem:0(0.000%) - Err:2541.809814
2022-04-23 01:52:43 - fullrmc <INFO> @0 Gen:2307571 - Tr:679034(29.426%) - Acc:10917(0.473%) - Rem:0(0.000%) - Err:2541.790283
2022-04-23 01:52:43 - fullrmc <INFO> @0 Gen:2307597 - Tr:679042(29.426%) - Acc:10918(0.473%) - Rem:0(0.000%) - Err:2541.787109
2022-04-23 01:52:44 - fullrmc <INFO> @0 Gen:2307892 - Tr:679137(29.427%) - Acc:10919(0.473%) - Rem:0(0.000%) - Err:2541.570312
2022-04-23 01:52:44 - fullrmc <INFO> @0 Gen:2308188 - Tr:679217(29.426%) - Acc:10920(0.473%) - Rem:0(0.000%) - Err:2541.402344
2022-04-23 01:52:45 - fullrmc <INFO> @0 Gen:2308378 - Tr:679270(29.426%) - Acc:10921(0.473%) - Rem:0(0.000%) - Err:2541.363281
2022-04-23 01:52:48 - fullrmc <INFO> @0 Gen:2310717 - Tr:679975(29.427%) - Acc:10922(0.473%) - Rem:0(0.000%) - 

2022-04-23 01:54:09 - fullrmc <INFO> @0 Gen:2357538 - Tr:693923(29.434%) - Acc:10979(0.466%) - Rem:0(0.000%) - Err:2537.676758
2022-04-23 01:54:11 - fullrmc <INFO> @0 Gen:2359201 - Tr:694405(29.434%) - Acc:10980(0.465%) - Rem:0(0.000%) - Err:2537.674316
2022-04-23 01:54:12 - fullrmc <INFO> @0 Gen:2359519 - Tr:694494(29.434%) - Acc:10981(0.465%) - Rem:0(0.000%) - Err:2537.657227
2022-04-23 01:54:12 - fullrmc <INFO> @0 Gen:2359704 - Tr:694548(29.434%) - Acc:10982(0.465%) - Rem:0(0.000%) - Err:2537.512695
2022-04-23 01:54:13 - fullrmc <INFO> @0 Gen:2360244 - Tr:694700(29.433%) - Acc:10983(0.465%) - Rem:0(0.000%) - Err:2537.468506
2022-04-23 01:54:19 - fullrmc <INFO> @0 Gen:2363739 - Tr:695722(29.433%) - Acc:10984(0.465%) - Rem:0(0.000%) - Err:2537.466553
2022-04-23 01:54:19 - fullrmc <INFO> @0 Gen:2364112 - Tr:695820(29.433%) - Acc:10985(0.465%) - Rem:0(0.000%) - Err:2537.424316
2022-04-23 01:54:20 - fullrmc <INFO> @0 Gen:2364700 - Tr:696004(29.433%) - Acc:10986(0.465%) - Rem:0(0.000%) - 

2022-04-23 01:55:49 - fullrmc <INFO> @0 Gen:2416980 - Tr:711539(29.439%) - Acc:11043(0.457%) - Rem:0(0.000%) - Err:2534.260742
2022-04-23 01:55:49 - fullrmc <INFO> @0 Gen:2417224 - Tr:711616(29.439%) - Acc:11044(0.457%) - Rem:0(0.000%) - Err:2534.260742
2022-04-23 01:55:51 - fullrmc <INFO> @0 Gen:2418209 - Tr:711901(29.439%) - Acc:11045(0.457%) - Rem:0(0.000%) - Err:2534.212402
2022-04-23 01:55:52 - fullrmc <INFO> @0 Gen:2418607 - Tr:712007(29.439%) - Acc:11046(0.457%) - Rem:0(0.000%) - Err:2533.831543
2022-04-23 01:55:52 - fullrmc <INFO> @0 Gen:2418682 - Tr:712025(29.439%) - Acc:11047(0.457%) - Rem:0(0.000%) - Err:2533.823242
2022-04-23 01:55:54 - fullrmc <INFO> @0 Gen:2420232 - Tr:712515(29.440%) - Acc:11048(0.456%) - Rem:0(0.000%) - Err:2533.808594
2022-04-23 01:55:55 - fullrmc <INFO> @0 Gen:2420673 - Tr:712644(29.440%) - Acc:11049(0.456%) - Rem:0(0.000%) - Err:2533.757080
2022-04-23 01:55:56 - fullrmc <INFO> @0 Gen:2421428 - Tr:712888(29.441%) - Acc:11050(0.456%) - Rem:0(0.000%) - 

...

2022-04-23 03:59:25 - fullrmc <INFO> @0 Gen:6774153 - Tr:2002639(29.563%) - Acc:14750(0.218%) - Rem:0(0.000%) - Err:2399.488525
2022-04-23 03:59:29 - fullrmc <INFO> @0 Gen:6776266 - Tr:2003263(29.563%) - Acc:14751(0.218%) - Rem:0(0.000%) - Err:2399.488525
2022-04-23 03:59:29 - fullrmc <INFO> @0 Gen:6776741 - Tr:2003419(29.563%) - Acc:14752(0.218%) - Rem:0(0.000%) - Err:2399.488525
2022-04-23 03:59:36 - fullrmc <INFO> @0 Gen:6780760 - Tr:2004631(29.564%) - Acc:14753(0.218%) - Rem:0(0.000%) - Err:2399.479492
2022-04-23 03:59:36 - fullrmc <INFO> @0 Gen:6780859 - Tr:2004666(29.564%) - Acc:14754(0.218%) - Rem:0(0.000%) - Err:2399.476562
2022-04-23 03:59:38 - fullrmc <INFO> @0 Gen:6782143 - Tr:2005049(29.564%) - Acc:14755(0.218%) - Rem:0(0.000%) - Err:2399.458252
2022-04-23 03:59:38 - fullrmc <INFO> @0 Gen:6782325 - Tr:2005099(29.564%) - Acc:14756(0.218%) - Rem:0(0.000%) - Err:2399.432129
2022-04-23 03:59:40 - fullrmc <INFO> @0 Gen:6783280 - Tr:2005371(29.563%) - Acc:14757(0.218%) - Rem:0(0.

2022-04-23 04:01:31 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 04:01:35 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 04:01:38 - fullrmc <INFO> @0 Gen:6851912 - Tr:2025765(29.565%) - Acc:14813(0.216%) - Rem:0(0.000%) - Err:2397.606689
2022-04-23 04:01:44 - fullrmc <INFO> @0 Gen:6856038 - Tr:2026965(29.565%) - Acc:14814(0.216%) - Rem:0(0.000%) - Err:2397.562988
2022-04-23 04:01:47 - fullrmc <INFO> @0 Gen:6857750 - Tr:2027463(29.565%) - Acc:14815(0.216%) - Rem:0(0.000%) - Err:2397.558594
2022-04-23 04:01:48 - fullrmc <INFO> @0 Gen:6858130 - Tr:2027577(29.565%) - Acc:14816(0.216%) - Rem:0(0.000%) - Err:2397.519043
2022-04-23 04:01:50 - fullrmc <INFO> @0 Gen:6859831 - Tr:2028067(29.564%) - Acc:14817(0.216%) - Rem:0(0.000%) - Err:2397.497559
2022-04-23 04:01:51 - fullrmc <INFO> @0 Gen:6860424 - Tr:2028250(29.564%) - Acc:14818(0.216%) - Rem:0(0.000%) - Err:2397.497559
2022-04-23 04:02:03 - fullrmc <INFO> @0 Gen:6867901 - Tr:2030542(29.566%) - Acc:14819(0.21

2022-04-23 04:04:12 - fullrmc <INFO> @0 Gen:6945171 - Tr:2053429(29.566%) - Acc:14875(0.214%) - Rem:0(0.000%) - Err:2396.363770
2022-04-23 04:04:13 - fullrmc <INFO> @0 Gen:6945493 - Tr:2053516(29.566%) - Acc:14876(0.214%) - Rem:0(0.000%) - Err:2396.359375
2022-04-23 04:04:19 - fullrmc <INFO> @0 Gen:6949439 - Tr:2054699(29.566%) - Acc:14877(0.214%) - Rem:0(0.000%) - Err:2396.357422
2022-04-23 04:04:19 - fullrmc <INFO> @0 Gen:6949621 - Tr:2054741(29.566%) - Acc:14878(0.214%) - Rem:0(0.000%) - Err:2396.323975
2022-04-23 04:04:20 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 04:04:24 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 04:04:27 - fullrmc <INFO> @0 Gen:6952243 - Tr:2055500(29.566%) - Acc:14879(0.214%) - Rem:0(0.000%) - Err:2396.323975
2022-04-23 04:04:33 - fullrmc <INFO> @0 Gen:6956021 - Tr:2056640(29.566%) - Acc:14880(0.214%) - Rem:0(0.000%) - Err:2396.312500
2022-04-23 04:04:35 - fullrmc <INFO> @0 Gen:6956987 - Tr:2056932(29.566%) - Acc:14881(0.21

2022-04-23 04:06:55 - fullrmc <INFO> @0 Gen:7042514 - Tr:2082362(29.568%) - Acc:14937(0.212%) - Rem:0(0.000%) - Err:2395.104736
2022-04-23 04:06:59 - fullrmc <INFO> @0 Gen:7044752 - Tr:2082983(29.568%) - Acc:14938(0.212%) - Rem:0(0.000%) - Err:2395.078857
2022-04-23 04:07:06 - fullrmc <INFO> @0 Gen:7049445 - Tr:2084373(29.568%) - Acc:14939(0.212%) - Rem:0(0.000%) - Err:2395.038330
2022-04-23 04:07:07 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 04:07:11 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 04:07:11 - fullrmc <INFO> @0 Gen:7050180 - Tr:2084594(29.568%) - Acc:14940(0.212%) - Rem:0(0.000%) - Err:2395.031738
2022-04-23 04:07:11 - fullrmc <INFO> @0 Gen:7050336 - Tr:2084648(29.568%) - Acc:14941(0.212%) - Rem:0(0.000%) - Err:2395.030518
2022-04-23 04:07:12 - fullrmc <INFO> @0 Gen:7050954 - Tr:2084833(29.568%) - Acc:14942(0.212%) - Rem:0(0.000%) - Err:2395.020508
2022-04-23 04:07:14 - fullrmc <INFO> @0 Gen:7052156 - Tr:2085223(29.569%) - Acc:14943(0.21

2022-04-23 04:09:37 - fullrmc <INFO> @0 Gen:7137658 - Tr:2110613(29.570%) - Acc:14999(0.210%) - Rem:0(0.000%) - Err:2394.042480
2022-04-23 04:09:38 - fullrmc <INFO> @0 Gen:7138413 - Tr:2110831(29.570%) - Acc:15000(0.210%) - Rem:0(0.000%) - Err:2394.006592
2022-04-23 04:09:38 - fullrmc <INFO> @0 Gen:7138740 - Tr:2110924(29.570%) - Acc:15001(0.210%) - Rem:0(0.000%) - Err:2393.957031
2022-04-23 04:09:40 - fullrmc <INFO> @0 Gen:7139813 - Tr:2111234(29.570%) - Acc:15002(0.210%) - Rem:0(0.000%) - Err:2393.936279
2022-04-23 04:09:43 - fullrmc <INFO> @0 Gen:7141934 - Tr:2111834(29.569%) - Acc:15003(0.210%) - Rem:0(0.000%) - Err:2393.907715
2022-04-23 04:09:44 - fullrmc <INFO> @0 Gen:7142229 - Tr:2111917(29.569%) - Acc:15004(0.210%) - Rem:0(0.000%) - Err:2393.890625
2022-04-23 04:09:44 - fullrmc <INFO> @0 Gen:7142242 - Tr:2111923(29.569%) - Acc:15005(0.210%) - Rem:0(0.000%) - Err:2393.890625
2022-04-23 04:09:45 - fullrmc <INFO> @0 Gen:7142849 - Tr:2112129(29.570%) - Acc:15006(0.210%) - Rem:0(0.

In [17]:
ENGINE.set_groups_as_atoms()
ENGINE.run(numberOfSteps=1000000, saveFrequency=10000)

2022-04-23 04:11:25 - fullrmc <INFO> Engine @0 started 1000000 steps, total standard error is: 2393.619385
2022-04-23 04:11:30 - fullrmc <INFO> @0 Gen:7203459 - Tr:2130208(29.572%) - Acc:15034(0.209%) - Rem:0(0.000%) - Err:2393.612061
2022-04-23 04:11:30 - fullrmc <INFO> @0 Gen:7203483 - Tr:2130215(29.572%) - Acc:15035(0.209%) - Rem:0(0.000%) - Err:2393.612061
2022-04-23 04:11:33 - fullrmc <INFO> @0 Gen:7205039 - Tr:2130710(29.572%) - Acc:15036(0.209%) - Rem:0(0.000%) - Err:2393.612061
2022-04-23 04:11:39 - fullrmc <INFO> @0 Gen:7208996 - Tr:2131863(29.572%) - Acc:15037(0.209%) - Rem:0(0.000%) - Err:2393.565430
2022-04-23 04:11:40 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 04:11:44 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 04:11:51 - fullrmc <INFO> @0 Gen:7214258 - Tr:2133413(29.572%) - Acc:15038(0.208%) - Rem:0(0.000%) - Err:2393.554932
2022-04-23 04:11:51 - fullrmc <INFO> @0 Gen:7214295 - Tr:2133424(29.572%) - Acc:15039(0.208%) - Rem:0(0.000%) -

2022-04-23 04:14:14 - fullrmc <INFO> @0 Gen:7287328 - Tr:2155381(29.577%) - Acc:15089(0.207%) - Rem:0(0.000%) - Err:2392.797607
2022-04-23 04:14:18 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 04:14:22 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 04:14:25 - fullrmc <INFO> @0 Gen:7291373 - Tr:2156607(29.578%) - Acc:15090(0.207%) - Rem:0(0.000%) - Err:2392.794922
2022-04-23 04:14:27 - fullrmc <INFO> @0 Gen:7293069 - Tr:2157132(29.578%) - Acc:15091(0.207%) - Rem:0(0.000%) - Err:2392.781250
2022-04-23 04:14:27 - fullrmc <INFO> @0 Gen:7293092 - Tr:2157139(29.578%) - Acc:15092(0.207%) - Rem:0(0.000%) - Err:2392.769043
2022-04-23 04:14:29 - fullrmc <INFO> @0 Gen:7293960 - Tr:2157404(29.578%) - Acc:15093(0.207%) - Rem:0(0.000%) - Err:2392.763184
2022-04-23 04:14:30 - fullrmc <INFO> @0 Gen:7294673 - Tr:2157593(29.578%) - Acc:15094(0.207%) - Rem:0(0.000%) - Err:2392.763184
2022-04-23 04:14:32 - fullrmc <INFO> @0 Gen:7296101 - Tr:2157982(29.577%) - Acc:15095(0.20

2022-04-23 04:17:29 - fullrmc <INFO> @0 Gen:7387242 - Tr:2185119(29.580%) - Acc:15143(0.205%) - Rem:0(0.000%) - Err:2392.201904
2022-04-23 04:17:30 - fullrmc <INFO> @0 Gen:7388094 - Tr:2185372(29.580%) - Acc:15144(0.205%) - Rem:0(0.000%) - Err:2392.174805
2022-04-23 04:17:33 - fullrmc <INFO> @0 Gen:7389427 - Tr:2185762(29.580%) - Acc:15145(0.205%) - Rem:0(0.000%) - Err:2392.174805
2022-04-23 04:17:34 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 04:17:37 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 04:17:38 - fullrmc <INFO> @0 Gen:7390549 - Tr:2186093(29.580%) - Acc:15146(0.205%) - Rem:0(0.000%) - Err:2392.169434
2022-04-23 04:17:41 - fullrmc <INFO> @0 Gen:7391941 - Tr:2186528(29.580%) - Acc:15147(0.205%) - Rem:0(0.000%) - Err:2392.168945
2022-04-23 04:17:42 - fullrmc <INFO> @0 Gen:7392929 - Tr:2186825(29.580%) - Acc:15148(0.205%) - Rem:0(0.000%) - Err:2392.159912
2022-04-23 04:17:45 - fullrmc <INFO> @0 Gen:7394832 - Tr:2187395(29.580%) - Acc:15149(0.20

2022-04-23 04:20:33 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 04:20:36 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 04:20:37 - fullrmc <INFO> @0 Gen:7480386 - Tr:2212996(29.584%) - Acc:15198(0.203%) - Rem:0(0.000%) - Err:2390.588135
2022-04-23 04:20:37 - fullrmc <INFO> @0 Gen:7480562 - Tr:2213053(29.584%) - Acc:15199(0.203%) - Rem:0(0.000%) - Err:2390.586914
2022-04-23 04:20:43 - fullrmc <INFO> @0 Gen:7483986 - Tr:2214046(29.584%) - Acc:15200(0.203%) - Rem:0(0.000%) - Err:2390.577637
2022-04-23 04:20:45 - fullrmc <INFO> @0 Gen:7485425 - Tr:2214468(29.584%) - Acc:15201(0.203%) - Rem:0(0.000%) - Err:2390.576660
2022-04-23 04:20:48 - fullrmc <INFO> @0 Gen:7487103 - Tr:2214976(29.584%) - Acc:15202(0.203%) - Rem:0(0.000%) - Err:2390.319092
2022-04-23 04:20:49 - fullrmc <INFO> @0 Gen:7487712 - Tr:2215169(29.584%) - Acc:15203(0.203%) - Rem:0(0.000%) - Err:2390.285889
2022-04-23 04:20:49 - fullrmc <INFO> @0 Gen:7487728 - Tr:2215171(29.584%) - Acc:15204(0.20

2022-04-23 04:22:47 - fullrmc <INFO> @0 Gen:7547056 - Tr:2233072(29.589%) - Acc:15255(0.202%) - Rem:0(0.000%) - Err:2388.646484
2022-04-23 04:22:50 - fullrmc <INFO> @0 Gen:7549103 - Tr:2233703(29.589%) - Acc:15256(0.202%) - Rem:0(0.000%) - Err:2388.622070
2022-04-23 04:22:52 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 04:22:56 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 04:22:57 - fullrmc <INFO> @0 Gen:7550450 - Tr:2234109(29.589%) - Acc:15257(0.202%) - Rem:0(0.000%) - Err:2388.619385
2022-04-23 04:22:58 - fullrmc <INFO> @0 Gen:7551100 - Tr:2234298(29.589%) - Acc:15258(0.202%) - Rem:0(0.000%) - Err:2388.595459
2022-04-23 04:23:01 - fullrmc <INFO> @0 Gen:7553344 - Tr:2234976(29.589%) - Acc:15259(0.202%) - Rem:0(0.000%) - Err:2388.544678
2022-04-23 04:23:02 - fullrmc <INFO> @0 Gen:7553811 - Tr:2235109(29.589%) - Acc:15260(0.202%) - Rem:0(0.000%) - Err:2388.525635
2022-04-23 04:23:06 - fullrmc <INFO> @0 Gen:7556475 - Tr:2235918(29.589%) - Acc:15261(0.20

...

2022-04-23 04:28:31 - fullrmc <INFO> @0 Gen:7719612 - Tr:2285013(29.600%) - Acc:15366(0.199%) - Rem:0(0.000%) - Err:2386.656494
2022-04-23 04:28:32 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 04:28:35 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 04:28:36 - fullrmc <INFO> @0 Gen:7720222 - Tr:2285192(29.600%) - Acc:15367(0.199%) - Rem:0(0.000%) - Err:2386.651367
2022-04-23 04:28:38 - fullrmc <INFO> @0 Gen:7721485 - Tr:2285576(29.600%) - Acc:15368(0.199%) - Rem:0(0.000%) - Err:2386.599854
2022-04-23 04:28:38 - fullrmc <INFO> @0 Gen:7721778 - Tr:2285670(29.600%) - Acc:15369(0.199%) - Rem:0(0.000%) - Err:2386.599854
2022-04-23 04:28:39 - fullrmc <INFO> @0 Gen:7722406 - Tr:2285859(29.600%) - Acc:15370(0.199%) - Rem:0(0.000%) - Err:2386.541016
2022-04-23 04:28:41 - fullrmc <INFO> @0 Gen:7723344 - Tr:2286142(29.600%) - Acc:15371(0.199%) - Rem:0(0.000%) - Err:2386.523193
2022-04-23 04:28:47 - fullrmc <INFO> @0 Gen:7727032 - Tr:2287286(29.601%) - Acc:15372(0.19

2022-04-23 04:31:09 - fullrmc <INFO> @0 Gen:7798755 - Tr:2308749(29.604%) - Acc:15422(0.198%) - Rem:0(0.000%) - Err:2385.301025
2022-04-23 04:31:11 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 04:31:15 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 04:31:17 - fullrmc <INFO> @0 Gen:7801869 - Tr:2309690(29.604%) - Acc:15423(0.198%) - Rem:0(0.000%) - Err:2385.271484
2022-04-23 04:31:18 - fullrmc <INFO> @0 Gen:7801958 - Tr:2309717(29.604%) - Acc:15424(0.198%) - Rem:0(0.000%) - Err:2385.271484
2022-04-23 04:31:19 - fullrmc <INFO> @0 Gen:7802572 - Tr:2309904(29.604%) - Acc:15425(0.198%) - Rem:0(0.000%) - Err:2385.243652
2022-04-23 04:31:20 - fullrmc <INFO> @0 Gen:7803633 - Tr:2310245(29.605%) - Acc:15426(0.198%) - Rem:0(0.000%) - Err:2385.222656
2022-04-23 04:31:21 - fullrmc <INFO> @0 Gen:7803641 - Tr:2310249(29.605%) - Acc:15427(0.198%) - Rem:0(0.000%) - Err:2385.208008
2022-04-23 04:31:22 - fullrmc <INFO> @0 Gen:7804250 - Tr:2310409(29.604%) - Acc:15428(0.19

2022-04-23 04:34:14 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 04:34:21 - fullrmc <INFO> @0 Gen:7894401 - Tr:2337449(29.609%) - Acc:15476(0.196%) - Rem:0(0.000%) - Err:2384.366211
2022-04-23 04:34:21 - fullrmc <INFO> @0 Gen:7894508 - Tr:2337479(29.609%) - Acc:15477(0.196%) - Rem:0(0.000%) - Err:2384.353271
2022-04-23 04:34:28 - fullrmc <INFO> @0 Gen:7898639 - Tr:2338769(29.610%) - Acc:15478(0.196%) - Rem:0(0.000%) - Err:2384.346924
2022-04-23 04:34:30 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 04:34:34 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 04:34:35 - fullrmc <INFO> @0 Gen:7900631 - Tr:2339350(29.610%) - Acc:15479(0.196%) - Rem:0(0.000%) - Err:2384.346924
2022-04-23 04:34:42 - fullrmc <INFO> @0 Gen:7905252 - Tr:2340714(29.610%) - Acc:15480(0.196%) - Rem:0(0.000%) - Err:2384.329102
2022-04-23 04:34:48 - fullrmc <INFO> @0 Gen:7908652 - Tr:2341695(29.609%) - Acc:15481(0.196%) - Rem:0(0.000%) - Err:2384.308594
2022-04-23 04:34:50 - f

2022-04-23 04:38:04 - fullrmc <INFO> @0 Gen:8007098 - Tr:2371257(29.614%) - Acc:15528(0.194%) - Rem:0(0.000%) - Err:2383.759766
2022-04-23 04:38:06 - fullrmc <INFO> @0 Gen:8008341 - Tr:2371622(29.614%) - Acc:15529(0.194%) - Rem:0(0.000%) - Err:2383.742432
2022-04-23 04:38:08 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 04:38:13 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 04:38:18 - fullrmc <INFO> @0 Gen:8013390 - Tr:2373148(29.615%) - Acc:15530(0.194%) - Rem:0(0.000%) - Err:2383.737793
2022-04-23 04:38:21 - fullrmc <INFO> @0 Gen:8015183 - Tr:2373696(29.615%) - Acc:15531(0.194%) - Rem:0(0.000%) - Err:2383.722900
2022-04-23 04:38:24 - fullrmc <INFO> @0 Gen:8017448 - Tr:2374332(29.615%) - Acc:15532(0.194%) - Rem:0(0.000%) - Err:2383.705811
2022-04-23 04:38:25 - fullrmc <INFO> @0 Gen:8017938 - Tr:2374476(29.615%) - Acc:15533(0.194%) - Rem:0(0.000%) - Err:2383.705811
2022-04-23 04:38:28 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 04:3

2022-04-23 04:41:49 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 04:41:49 - fullrmc <INFO> @0 Gen:8120145 - Tr:2405100(29.619%) - Acc:15580(0.192%) - Rem:0(0.000%) - Err:2383.328613
2022-04-23 04:41:49 - fullrmc <INFO> @0 Gen:8120150 - Tr:2405103(29.619%) - Acc:15581(0.192%) - Rem:0(0.000%) - Err:2383.325195
2022-04-23 04:41:52 - fullrmc <INFO> @0 Gen:8121892 - Tr:2405658(29.619%) - Acc:15582(0.192%) - Rem:0(0.000%) - Err:2383.316895
2022-04-23 04:42:02 - fullrmc <INFO> @0 Gen:8128144 - Tr:2407582(29.620%) - Acc:15583(0.192%) - Rem:0(0.000%) - Err:2383.305420
2022-04-23 04:42:05 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 04:42:08 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 04:42:11 - fullrmc <INFO> @0 Gen:8131307 - Tr:2408528(29.620%) - Acc:15584(0.192%) - Rem:0(0.000%) - Err:2383.305420
2022-04-23 04:42:16 - fullrmc <INFO> @0 Gen:8134940 - Tr:2409584(29.620%) - Acc:15585(0.192%) - Rem:0(0.000%) - Err:2383.289795
2022-04-23 04:42:20 - f

In [18]:
ENGINE.set_groups_as_atoms()
ENGINE.run(numberOfSteps=5000000, saveFrequency=50000)

2022-04-23 04:44:23 - fullrmc <INFO> Engine @0 started 5000000 steps, total standard error is: 2383.065186
2022-04-23 04:44:34 - fullrmc <INFO> @0 Gen:8206972 - Tr:2431191(29.623%) - Acc:15615(0.190%) - Rem:0(0.000%) - Err:2383.037354
2022-04-23 04:44:35 - fullrmc <INFO> @0 Gen:8207123 - Tr:2431234(29.623%) - Acc:15616(0.190%) - Rem:0(0.000%) - Err:2383.028320
2022-04-23 04:44:49 - fullrmc <INFO> @0 Gen:8216433 - Tr:2434016(29.624%) - Acc:15617(0.190%) - Rem:0(0.000%) - Err:2383.016846
2022-04-23 04:44:50 - fullrmc <INFO> @0 Gen:8217233 - Tr:2434263(29.624%) - Acc:15618(0.190%) - Rem:0(0.000%) - Err:2383.016357
2022-04-23 04:44:52 - fullrmc <INFO> @0 Gen:8218368 - Tr:2434577(29.624%) - Acc:15619(0.190%) - Rem:0(0.000%) - Err:2383.016357
2022-04-23 04:44:53 - fullrmc <INFO> @0 Gen:8218995 - Tr:2434757(29.624%) - Acc:15620(0.190%) - Rem:0(0.000%) - Err:2382.965088
2022-04-23 04:44:56 - fullrmc <INFO> @0 Gen:8220759 - Tr:2435271(29.623%) - Acc:15621(0.190%) - Rem:0(0.000%) - Err:2382.9550

2022-04-23 04:48:57 - fullrmc <INFO> @0 Gen:8364797 - Tr:2478617(29.632%) - Acc:15675(0.187%) - Rem:0(0.000%) - Err:2382.436035
2022-04-23 04:48:59 - fullrmc <INFO> @0 Gen:8365780 - Tr:2478947(29.632%) - Acc:15676(0.187%) - Rem:0(0.000%) - Err:2382.420410
2022-04-23 04:49:01 - fullrmc <INFO> @0 Gen:8367004 - Tr:2479329(29.632%) - Acc:15677(0.187%) - Rem:0(0.000%) - Err:2382.420410
2022-04-23 04:49:04 - fullrmc <INFO> @0 Gen:8368992 - Tr:2479905(29.632%) - Acc:15678(0.187%) - Rem:0(0.000%) - Err:2382.402832
2022-04-23 04:49:08 - fullrmc <INFO> @0 Gen:8371821 - Tr:2480747(29.632%) - Acc:15679(0.187%) - Rem:0(0.000%) - Err:2382.366455
2022-04-23 04:49:10 - fullrmc <INFO> @0 Gen:8372762 - Tr:2481015(29.632%) - Acc:15680(0.187%) - Rem:0(0.000%) - Err:2382.366455
2022-04-23 04:49:14 - fullrmc <INFO> @0 Gen:8375530 - Tr:2481838(29.632%) - Acc:15681(0.187%) - Rem:0(0.000%) - Err:2382.364502
2022-04-23 04:49:17 - fullrmc <INFO> @0 Gen:8377440 - Tr:2482418(29.632%) - Acc:15682(0.187%) - Rem:0(0.

2022-04-23 04:53:09 - fullrmc <INFO> @0 Gen:8516840 - Tr:2524270(29.639%) - Acc:15736(0.185%) - Rem:0(0.000%) - Err:2381.736084
2022-04-23 04:53:12 - fullrmc <INFO> @0 Gen:8518528 - Tr:2524796(29.639%) - Acc:15737(0.185%) - Rem:0(0.000%) - Err:2381.718018
2022-04-23 04:53:13 - fullrmc <INFO> @0 Gen:8519691 - Tr:2525144(29.639%) - Acc:15738(0.185%) - Rem:0(0.000%) - Err:2381.710449
2022-04-23 04:53:14 - fullrmc <INFO> @0 Gen:8519962 - Tr:2525220(29.639%) - Acc:15739(0.185%) - Rem:0(0.000%) - Err:2381.710205
2022-04-23 04:53:14 - fullrmc <INFO> @0 Gen:8520264 - Tr:2525306(29.639%) - Acc:15740(0.185%) - Rem:0(0.000%) - Err:2381.702393
2022-04-23 04:53:17 - fullrmc <INFO> @0 Gen:8521955 - Tr:2525807(29.639%) - Acc:15741(0.185%) - Rem:0(0.000%) - Err:2381.702393
2022-04-23 04:53:21 - fullrmc <INFO> @0 Gen:8524214 - Tr:2526468(29.639%) - Acc:15742(0.185%) - Rem:0(0.000%) - Err:2381.695557
2022-04-23 04:53:22 - fullrmc <INFO> @0 Gen:8525184 - Tr:2526768(29.639%) - Acc:15743(0.185%) - Rem:0(0.

2022-04-23 04:56:38 - fullrmc <INFO> @0 Gen:8643487 - Tr:2562199(29.643%) - Acc:15798(0.183%) - Rem:0(0.000%) - Err:2381.224121
2022-04-23 04:56:42 - fullrmc <INFO> @0 Gen:8646439 - Tr:2563083(29.643%) - Acc:15799(0.183%) - Rem:0(0.000%) - Err:2381.206299
2022-04-23 04:56:45 - fullrmc <INFO> @0 Gen:8648091 - Tr:2563574(29.643%) - Acc:15800(0.183%) - Rem:0(0.000%) - Err:2381.206299
2022-04-23 04:56:48 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 04:56:52 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 04:56:56 - fullrmc <INFO> @0 Gen:8652352 - Tr:2564860(29.644%) - Acc:15801(0.183%) - Rem:0(0.000%) - Err:2381.189941
2022-04-23 04:57:02 - fullrmc <INFO> @0 Gen:8656287 - Tr:2566050(29.644%) - Acc:15802(0.183%) - Rem:0(0.000%) - Err:2381.188232
2022-04-23 04:57:07 - fullrmc <INFO> @0 Gen:8659441 - Tr:2567004(29.644%) - Acc:15803(0.182%) - Rem:0(0.000%) - Err:2381.188232
2022-04-23 04:57:16 - fullrmc <INFO> @0 Gen:8664968 - Tr:2568618(29.644%) - Acc:15804(0.18

2022-04-23 05:01:53 - fullrmc <INFO> @0 Gen:8832950 - Tr:2618961(29.650%) - Acc:15858(0.180%) - Rem:0(0.000%) - Err:2380.741943
2022-04-23 05:01:55 - fullrmc <INFO> @0 Gen:8834656 - Tr:2619478(29.650%) - Acc:15859(0.180%) - Rem:0(0.000%) - Err:2380.741943
2022-04-23 05:01:55 - fullrmc <INFO> @0 Gen:8834664 - Tr:2619479(29.650%) - Acc:15860(0.180%) - Rem:0(0.000%) - Err:2380.735107
2022-04-23 05:02:04 - fullrmc <INFO> @0 Gen:8840336 - Tr:2621192(29.650%) - Acc:15861(0.179%) - Rem:0(0.000%) - Err:2380.730957
2022-04-23 05:02:04 - fullrmc <INFO> @0 Gen:8840350 - Tr:2621196(29.650%) - Acc:15862(0.179%) - Rem:0(0.000%) - Err:2380.725586
2022-04-23 05:02:12 - fullrmc <INFO> @0 Gen:8845258 - Tr:2622654(29.650%) - Acc:15863(0.179%) - Rem:0(0.000%) - Err:2380.725586
2022-04-23 05:02:17 - fullrmc <INFO> @0 Gen:8848392 - Tr:2623631(29.651%) - Acc:15864(0.179%) - Rem:0(0.000%) - Err:2380.725586
2022-04-23 05:02:20 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 05:02:23 - fullrmc

...

2022-04-23 06:33:06 - fullrmc <INFO> @0 Gen:12125222 - Tr:3603460(29.719%) - Acc:16943(0.140%) - Rem:0(0.000%) - Err:2371.593750
2022-04-23 06:33:14 - fullrmc <INFO> @0 Gen:12130648 - Tr:3605087(29.719%) - Acc:16944(0.140%) - Rem:0(0.000%) - Err:2371.593750
2022-04-23 06:33:18 - fullrmc <INFO> @0 Gen:12132783 - Tr:3605701(29.719%) - Acc:16945(0.140%) - Rem:0(0.000%) - Err:2371.590820
2022-04-23 06:33:27 - fullrmc <INFO> @0 Gen:12138771 - Tr:3607538(29.719%) - Acc:16946(0.140%) - Rem:0(0.000%) - Err:2371.588867
2022-04-23 06:33:29 - fullrmc <INFO> @0 Gen:12139813 - Tr:3607862(29.719%) - Acc:16947(0.140%) - Rem:0(0.000%) - Err:2371.576904
2022-04-23 06:33:40 - fullrmc <INFO> @0 Gen:12146673 - Tr:3609900(29.719%) - Acc:16948(0.140%) - Rem:0(0.000%) - Err:2371.576904
2022-04-23 06:33:45 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 06:33:49 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 06:33:53 - fullrmc <INFO> @0 Gen:12152838 - Tr:3611689(29.719%) - Acc:169

2022-04-23 06:39:37 - fullrmc <INFO> @0 Gen:12357003 - Tr:3672767(29.722%) - Acc:17002(0.138%) - Rem:0(0.000%) - Err:2371.396240
2022-04-23 06:39:38 - fullrmc <INFO> @0 Gen:12357681 - Tr:3672979(29.722%) - Acc:17003(0.138%) - Rem:0(0.000%) - Err:2371.396240
2022-04-23 06:39:45 - fullrmc <INFO> @0 Gen:12362188 - Tr:3674340(29.722%) - Acc:17004(0.138%) - Rem:0(0.000%) - Err:2371.391357
2022-04-23 06:39:55 - fullrmc <INFO> @0 Gen:12368562 - Tr:3676256(29.723%) - Acc:17005(0.137%) - Rem:0(0.000%) - Err:2371.391357
2022-04-23 06:39:55 - fullrmc <INFO> @0 Gen:12368739 - Tr:3676301(29.723%) - Acc:17006(0.137%) - Rem:0(0.000%) - Err:2371.391357
2022-04-23 06:39:56 - fullrmc <INFO> @0 Gen:12368829 - Tr:3676330(29.723%) - Acc:17007(0.137%) - Rem:0(0.000%) - Err:2371.391357
2022-04-23 06:40:00 - fullrmc <INFO> @0 Gen:12371562 - Tr:3677111(29.722%) - Acc:17008(0.137%) - Rem:0(0.000%) - Err:2371.391357
2022-04-23 06:40:05 - fullrmc <INFO> @0 Gen:12374872 - Tr:3678119(29.722%) - Acc:17009(0.137%) - 

2022-04-23 06:47:41 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 06:47:57 - fullrmc <INFO> @0 Gen:12660144 - Tr:3763507(29.727%) - Acc:17060(0.135%) - Rem:0(0.000%) - Err:2371.283936
2022-04-23 06:48:03 - fullrmc <INFO> @0 Gen:12663717 - Tr:3764545(29.727%) - Acc:17061(0.135%) - Rem:0(0.000%) - Err:2371.270020
2022-04-23 06:48:22 - fullrmc <INFO> @0 Gen:12676668 - Tr:3768348(29.727%) - Acc:17062(0.135%) - Rem:0(0.000%) - Err:2371.267822
2022-04-23 06:48:25 - fullrmc <INFO> @0 Gen:12678525 - Tr:3768897(29.727%) - Acc:17063(0.135%) - Rem:0(0.000%) - Err:2371.267822
2022-04-23 06:48:26 - fullrmc <INFO> @0 Gen:12678858 - Tr:3768999(29.727%) - Acc:17064(0.135%) - Rem:0(0.000%) - Err:2371.267822
2022-04-23 06:48:27 - fullrmc <INFO> @0 Gen:12679807 - Tr:3769287(29.727%) - Acc:17065(0.135%) - Rem:0(0.000%) - Err:2371.267822
2022-04-23 06:48:52 - fullrmc <INFO> @0 Gen:12695629 - Tr:3774109(29.728%) - Acc:17066(0.134%) - Rem:0(0.000%) - Err:2371.267334
2022-04-23 06:48:53 - fullrmc 

2022-04-23 06:55:48 - fullrmc <INFO> @0 Gen:12947730 - Tr:3849176(29.729%) - Acc:17118(0.132%) - Rem:0(0.000%) - Err:2371.138672
2022-04-23 06:55:52 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 06:55:56 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 06:56:04 - fullrmc <INFO> @0 Gen:12955051 - Tr:3851257(29.728%) - Acc:17119(0.132%) - Rem:0(0.000%) - Err:2371.137451
2022-04-23 06:56:06 - fullrmc <INFO> @0 Gen:12956574 - Tr:3851714(29.728%) - Acc:17120(0.132%) - Rem:0(0.000%) - Err:2371.137451
2022-04-23 06:56:12 - fullrmc <INFO> @0 Gen:12960454 - Tr:3852853(29.728%) - Acc:17121(0.132%) - Rem:0(0.000%) - Err:2371.137451
2022-04-23 06:56:33 - fullrmc <INFO> @0 Gen:12973245 - Tr:3856603(29.727%) - Acc:17122(0.132%) - Rem:0(0.000%) - Err:2371.136719
2022-04-23 06:56:34 - fullrmc <INFO> @0 Gen:12974085 - Tr:3856859(29.727%) - Acc:17123(0.132%) - Rem:0(0.000%) - Err:2371.136719
2022-04-23 06:56:49 - fullrmc <INFO> @0 Gen:12984170 - Tr:3859874(29.728%) - Acc:171

2022-04-23 07:02:06 - fullrmc <INFO> @0 Gen:13174538 - Tr:3916864(29.731%) - Acc:17177(0.130%) - Rem:0(0.000%) - Err:2370.993652
2022-04-23 07:02:20 - fullrmc <INFO> @0 Gen:13183265 - Tr:3919497(29.731%) - Acc:17178(0.130%) - Rem:0(0.000%) - Err:2370.993652
2022-04-23 07:02:28 - fullrmc <INFO> @0 Gen:13188534 - Tr:3921033(29.731%) - Acc:17179(0.130%) - Rem:0(0.000%) - Err:2370.993652
2022-04-23 07:02:37 - fullrmc <INFO> @0 Gen:13194007 - Tr:3922743(29.731%) - Acc:17180(0.130%) - Rem:0(0.000%) - Err:2370.993652
2022-04-23 07:02:39 - fullrmc <INFO> @0 Gen:13195725 - Tr:3923246(29.731%) - Acc:17181(0.130%) - Rem:0(0.000%) - Err:2370.989990
2022-04-23 07:02:46 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 07:02:50 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 07:02:50 - pdbparser <INFO> All records successfully exported to 'restart.pdb'
2022-04-23 07:02:50 - fullrmc <INFO> Engine @0 finishes executing all '5000000' steps in 0(days) 2:18:26


In [19]:
ENGINE.set_groups_as_atoms()
ENGINE.run(numberOfSteps=10000000, saveFrequency=100000)

2022-04-23 07:02:51 - fullrmc <INFO> Engine @0 started 10000000 steps, total standard error is: 2370.989990
2022-04-23 07:02:51 - fullrmc <INFO> @0 Gen:13200025 - Tr:3924513(29.731%) - Acc:17182(0.130%) - Rem:0(0.000%) - Err:2370.949219
2022-04-23 07:02:55 - fullrmc <INFO> @0 Gen:13202712 - Tr:3925294(29.731%) - Acc:17183(0.130%) - Rem:0(0.000%) - Err:2370.947754
2022-04-23 07:02:57 - fullrmc <INFO> @0 Gen:13204005 - Tr:3925685(29.731%) - Acc:17184(0.130%) - Rem:0(0.000%) - Err:2370.944336
2022-04-23 07:03:06 - fullrmc <INFO> @0 Gen:13209880 - Tr:3927425(29.731%) - Acc:17185(0.130%) - Rem:0(0.000%) - Err:2370.944336
2022-04-23 07:03:09 - fullrmc <INFO> @0 Gen:13211955 - Tr:3928055(29.731%) - Acc:17186(0.130%) - Rem:0(0.000%) - Err:2370.940430
2022-04-23 07:03:12 - fullrmc <INFO> @0 Gen:13213403 - Tr:3928480(29.731%) - Acc:17187(0.130%) - Rem:0(0.000%) - Err:2370.936035
2022-04-23 07:03:13 - fullrmc <INFO> @0 Gen:13214000 - Tr:3928657(29.731%) - Acc:17188(0.130%) - Rem:0(0.000%) - Err:2

2022-04-23 07:08:57 - fullrmc <INFO> @0 Gen:13427209 - Tr:3992576(29.735%) - Acc:17243(0.128%) - Rem:0(0.000%) - Err:2370.858398
2022-04-23 07:09:02 - fullrmc <INFO> @0 Gen:13430326 - Tr:3993514(29.735%) - Acc:17244(0.128%) - Rem:0(0.000%) - Err:2370.858398
2022-04-23 07:09:07 - fullrmc <INFO> @0 Gen:13432822 - Tr:3994241(29.735%) - Acc:17245(0.128%) - Rem:0(0.000%) - Err:2370.858398
2022-04-23 07:09:13 - fullrmc <INFO> @0 Gen:13436095 - Tr:3995274(29.735%) - Acc:17246(0.128%) - Rem:0(0.000%) - Err:2370.857910
2022-04-23 07:09:16 - fullrmc <INFO> @0 Gen:13437929 - Tr:3995832(29.735%) - Acc:17247(0.128%) - Rem:0(0.000%) - Err:2370.853271
2022-04-23 07:09:29 - fullrmc <INFO> @0 Gen:13446107 - Tr:3998359(29.736%) - Acc:17248(0.128%) - Rem:0(0.000%) - Err:2370.852295
2022-04-23 07:09:37 - fullrmc <INFO> @0 Gen:13450908 - Tr:3999798(29.736%) - Acc:17249(0.128%) - Rem:0(0.000%) - Err:2370.852295
2022-04-23 07:09:38 - fullrmc <INFO> @0 Gen:13451982 - Tr:4000106(29.736%) - Acc:17250(0.128%) - 

2022-04-23 07:14:52 - fullrmc <INFO> @0 Gen:13645329 - Tr:4057881(29.738%) - Acc:17305(0.127%) - Rem:0(0.000%) - Err:2370.744385
2022-04-23 07:14:54 - fullrmc <INFO> @0 Gen:13646296 - Tr:4058173(29.738%) - Acc:17306(0.127%) - Rem:0(0.000%) - Err:2370.744385
2022-04-23 07:14:56 - fullrmc <INFO> @0 Gen:13647324 - Tr:4058446(29.738%) - Acc:17307(0.127%) - Rem:0(0.000%) - Err:2370.744385
2022-04-23 07:15:02 - fullrmc <INFO> @0 Gen:13650143 - Tr:4059281(29.738%) - Acc:17308(0.127%) - Rem:0(0.000%) - Err:2370.744385
2022-04-23 07:15:05 - fullrmc <INFO> @0 Gen:13652323 - Tr:4059930(29.738%) - Acc:17309(0.127%) - Rem:0(0.000%) - Err:2370.744385
2022-04-23 07:15:09 - fullrmc <INFO> @0 Gen:13654537 - Tr:4060596(29.738%) - Acc:17310(0.127%) - Rem:0(0.000%) - Err:2370.744385
2022-04-23 07:15:13 - fullrmc <INFO> @0 Gen:13657048 - Tr:4061330(29.738%) - Acc:17311(0.127%) - Rem:0(0.000%) - Err:2370.744385
2022-04-23 07:15:21 - fullrmc <INFO> @0 Gen:13662259 - Tr:4062926(29.738%) - Acc:17312(0.127%) - 

2022-04-23 07:21:24 - fullrmc <INFO> @0 Gen:13888980 - Tr:4130543(29.740%) - Acc:17367(0.125%) - Rem:0(0.000%) - Err:2370.634766
2022-04-23 07:21:33 - fullrmc <INFO> @0 Gen:13894205 - Tr:4132100(29.740%) - Acc:17368(0.125%) - Rem:0(0.000%) - Err:2370.634766
2022-04-23 07:21:41 - fullrmc <INFO> @0 Gen:13899638 - Tr:4133725(29.740%) - Acc:17369(0.125%) - Rem:0(0.000%) - Err:2370.634766
2022-04-23 07:21:42 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 07:21:46 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 07:22:04 - fullrmc <INFO> @0 Gen:13911451 - Tr:4137198(29.740%) - Acc:17370(0.125%) - Rem:0(0.000%) - Err:2370.633301
2022-04-23 07:22:05 - fullrmc <INFO> @0 Gen:13912381 - Tr:4137495(29.740%) - Acc:17371(0.125%) - Rem:0(0.000%) - Err:2370.626465
2022-04-23 07:22:17 - fullrmc <INFO> @0 Gen:13919726 - Tr:4139642(29.739%) - Acc:17372(0.125%) - Rem:0(0.000%) - Err:2370.626465
2022-04-23 07:22:28 - fullrmc <INFO> @0 Gen:13927381 - Tr:4141953(29.740%) - Acc:173

2022-04-23 07:29:18 - fullrmc <INFO> @0 Gen:14182582 - Tr:4218212(29.742%) - Acc:17428(0.123%) - Rem:0(0.000%) - Err:2370.518311
2022-04-23 07:29:28 - fullrmc <INFO> @0 Gen:14189292 - Tr:4220188(29.742%) - Acc:17429(0.123%) - Rem:0(0.000%) - Err:2370.518311
2022-04-23 07:29:38 - fullrmc <INFO> @0 Gen:14195819 - Tr:4222215(29.743%) - Acc:17430(0.123%) - Rem:0(0.000%) - Err:2370.518311
2022-04-23 07:29:45 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 07:29:49 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 07:29:53 - fullrmc <INFO> @0 Gen:14203047 - Tr:4224423(29.743%) - Acc:17431(0.123%) - Rem:0(0.000%) - Err:2370.512207
2022-04-23 07:29:54 - fullrmc <INFO> @0 Gen:14203437 - Tr:4224549(29.743%) - Acc:17432(0.123%) - Rem:0(0.000%) - Err:2370.502197
2022-04-23 07:29:55 - fullrmc <INFO> @0 Gen:14204144 - Tr:4224771(29.743%) - Acc:17433(0.123%) - Rem:0(0.000%) - Err:2370.499023
2022-04-23 07:29:57 - fullrmc <INFO> @0 Gen:14204966 - Tr:4225044(29.743%) - Acc:174

...

2022-04-23 11:05:14 - fullrmc <INFO> @0 Gen:22170627 - Tr:6604391(29.789%) - Acc:19908(0.090%) - Rem:0(0.000%) - Err:2352.392334
2022-04-23 11:05:15 - fullrmc <INFO> @0 Gen:22170849 - Tr:6604468(29.789%) - Acc:19909(0.090%) - Rem:0(0.000%) - Err:2352.392334
2022-04-23 11:05:22 - fullrmc <INFO> @0 Gen:22175356 - Tr:6605792(29.789%) - Acc:19910(0.090%) - Rem:0(0.000%) - Err:2352.378174
2022-04-23 11:05:29 - fullrmc <INFO> @0 Gen:22180177 - Tr:6607256(29.789%) - Acc:19911(0.090%) - Rem:0(0.000%) - Err:2352.377930
2022-04-23 11:05:48 - fullrmc <INFO> @0 Gen:22192103 - Tr:6610808(29.789%) - Acc:19912(0.090%) - Rem:0(0.000%) - Err:2352.371826
2022-04-23 11:05:51 - fullrmc <INFO> @0 Gen:22194175 - Tr:6611408(29.789%) - Acc:19913(0.090%) - Rem:0(0.000%) - Err:2352.371826
2022-04-23 11:05:55 - fullrmc <INFO> @0 Gen:22196918 - Tr:6612238(29.789%) - Acc:19914(0.090%) - Rem:0(0.000%) - Err:2352.371826
2022-04-23 11:05:57 - fullrmc <INFO> @0 Gen:22197726 - Tr:6612473(29.789%) - Acc:19915(0.090%) - 

2022-04-23 11:11:25 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 11:11:29 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 11:11:38 - fullrmc <INFO> @0 Gen:22405328 - Tr:6674883(29.791%) - Acc:19970(0.089%) - Rem:0(0.000%) - Err:2351.584473
2022-04-23 11:11:38 - fullrmc <INFO> @0 Gen:22405682 - Tr:6674975(29.791%) - Acc:19971(0.089%) - Rem:0(0.000%) - Err:2351.584473
2022-04-23 11:11:49 - fullrmc <INFO> @0 Gen:22412090 - Tr:6676846(29.791%) - Acc:19972(0.089%) - Rem:0(0.000%) - Err:2351.579346
2022-04-23 11:11:50 - fullrmc <INFO> @0 Gen:22412499 - Tr:6676971(29.791%) - Acc:19973(0.089%) - Rem:0(0.000%) - Err:2351.579346
2022-04-23 11:11:56 - fullrmc <INFO> @0 Gen:22416298 - Tr:6678117(29.791%) - Acc:19974(0.089%) - Rem:0(0.000%) - Err:2351.575439
2022-04-23 11:11:56 - fullrmc <INFO> @0 Gen:22416422 - Tr:6678161(29.791%) - Acc:19975(0.089%) - Rem:0(0.000%) - Err:2351.561035
2022-04-23 11:11:56 - fullrmc <INFO> @0 Gen:22416585 - Tr:6678211(29.791%) - Acc:199

2022-04-23 11:16:37 - fullrmc <INFO> @0 Gen:22589153 - Tr:6729672(29.792%) - Acc:20032(0.089%) - Rem:0(0.000%) - Err:2351.269531
2022-04-23 11:16:37 - fullrmc <INFO> @0 Gen:22589392 - Tr:6729743(29.792%) - Acc:20033(0.089%) - Rem:0(0.000%) - Err:2351.269531
2022-04-23 11:16:44 - fullrmc <INFO> @0 Gen:22593449 - Tr:6730998(29.792%) - Acc:20034(0.089%) - Rem:0(0.000%) - Err:2351.269531
2022-04-23 11:16:44 - fullrmc <INFO> @0 Gen:22593543 - Tr:6731022(29.792%) - Acc:20035(0.089%) - Rem:0(0.000%) - Err:2351.267090
2022-04-23 11:16:45 - fullrmc <INFO> @0 Gen:22593836 - Tr:6731109(29.792%) - Acc:20036(0.089%) - Rem:0(0.000%) - Err:2351.266113
2022-04-23 11:16:46 - fullrmc <INFO> @0 Gen:22594895 - Tr:6731412(29.792%) - Acc:20037(0.089%) - Rem:0(0.000%) - Err:2351.259033
2022-04-23 11:16:49 - fullrmc <INFO> @0 Gen:22596705 - Tr:6731973(29.792%) - Acc:20038(0.089%) - Rem:0(0.000%) - Err:2351.259033
2022-04-23 11:16:54 - fullrmc <INFO> @0 Gen:22599493 - Tr:6732794(29.792%) - Acc:20039(0.089%) - 

2022-04-23 11:21:28 - fullrmc <INFO> @0 Gen:22768252 - Tr:6783163(29.792%) - Acc:20094(0.088%) - Rem:0(0.000%) - Err:2351.065918
2022-04-23 11:21:36 - fullrmc <INFO> @0 Gen:22772969 - Tr:6784555(29.792%) - Acc:20095(0.088%) - Rem:0(0.000%) - Err:2351.065918
2022-04-23 11:21:41 - fullrmc <INFO> @0 Gen:22776119 - Tr:6785483(29.792%) - Acc:20096(0.088%) - Rem:0(0.000%) - Err:2351.065918
2022-04-23 11:21:41 - fullrmc <INFO> @0 Gen:22776360 - Tr:6785547(29.792%) - Acc:20097(0.088%) - Rem:0(0.000%) - Err:2351.065918
2022-04-23 11:22:13 - fullrmc <INFO> @0 Gen:22796845 - Tr:6791561(29.792%) - Acc:20098(0.088%) - Rem:0(0.000%) - Err:2351.063232
2022-04-23 11:22:13 - fullrmc <INFO> @0 Gen:22796951 - Tr:6791590(29.792%) - Acc:20099(0.088%) - Rem:0(0.000%) - Err:2351.049561
2022-04-23 11:22:13 - fullrmc <INFO> @0 Gen:22797165 - Tr:6791656(29.792%) - Acc:20100(0.088%) - Rem:0(0.000%) - Err:2351.046631
2022-04-23 11:22:18 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 11:22:21 - 

2022-04-23 11:27:42 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 11:27:45 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 11:27:49 - fullrmc <INFO> @0 Gen:23002339 - Tr:6852555(29.791%) - Acc:20156(0.088%) - Rem:0(0.000%) - Err:2350.822754
2022-04-23 11:27:55 - fullrmc <INFO> @0 Gen:23006012 - Tr:6853671(29.791%) - Acc:20157(0.088%) - Rem:0(0.000%) - Err:2350.822754
2022-04-23 11:27:55 - fullrmc <INFO> @0 Gen:23006320 - Tr:6853761(29.791%) - Acc:20158(0.088%) - Rem:0(0.000%) - Err:2350.822754
2022-04-23 11:27:57 - fullrmc <INFO> @0 Gen:23007254 - Tr:6854061(29.791%) - Acc:20159(0.088%) - Rem:0(0.000%) - Err:2350.822754
2022-04-23 11:27:59 - fullrmc <INFO> @0 Gen:23008351 - Tr:6854361(29.791%) - Acc:20160(0.088%) - Rem:0(0.000%) - Err:2350.821777
2022-04-23 11:28:15 - fullrmc <INFO> @0 Gen:23018810 - Tr:6857452(29.791%) - Acc:20161(0.088%) - Rem:0(0.000%) - Err:2350.817139
2022-04-23 11:28:30 - fullrmc <INFO> @0 Gen:23028364 - Tr:6860317(29.791%) - Acc:201

In [20]:
ENGINE.set_groups_as_atoms()
ENGINE.run(numberOfSteps=10000000, saveFrequency=100000)

2022-04-23 13:25:33 - fullrmc <INFO> Engine @0 started 10000000 steps, total standard error is: 2350.718262
2022-04-23 13:25:33 - fullrmc <INFO> @0 Gen:23200338 - Tr:6911460(29.790%) - Acc:20198(0.087%) - Rem:0(0.000%) - Err:2350.718262
2022-04-23 13:25:35 - fullrmc <INFO> @0 Gen:23201462 - Tr:6911809(29.790%) - Acc:20199(0.087%) - Rem:0(0.000%) - Err:2350.718262
2022-04-23 13:25:35 - fullrmc <INFO> @0 Gen:23201489 - Tr:6911816(29.790%) - Acc:20200(0.087%) - Rem:0(0.000%) - Err:2350.714844
2022-04-23 13:25:51 - fullrmc <INFO> @0 Gen:23210897 - Tr:6914656(29.791%) - Acc:20201(0.087%) - Rem:0(0.000%) - Err:2350.707520
2022-04-23 13:25:52 - fullrmc <INFO> @0 Gen:23211429 - Tr:6914826(29.791%) - Acc:20202(0.087%) - Rem:0(0.000%) - Err:2350.678711
2022-04-23 13:26:02 - fullrmc <INFO> @0 Gen:23214808 - Tr:6915862(29.791%) - Acc:20203(0.087%) - Rem:0(0.000%) - Err:2350.678711
2022-04-23 13:26:02 - fullrmc <INFO> @0 Gen:23214992 - Tr:6915903(29.791%) - Acc:20204(0.087%) - Rem:0(0.000%) - Err:2

2022-04-23 13:32:33 - fullrmc <INFO> @0 Gen:23394133 - Tr:6969629(29.792%) - Acc:20260(0.087%) - Rem:0(0.000%) - Err:2350.481201
2022-04-23 13:32:46 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 13:32:49 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 13:33:12 - fullrmc <INFO> @0 Gen:23409886 - Tr:6974350(29.792%) - Acc:20261(0.087%) - Rem:0(0.000%) - Err:2350.480225
2022-04-23 13:33:20 - fullrmc <INFO> @0 Gen:23413689 - Tr:6975499(29.792%) - Acc:20262(0.087%) - Rem:0(0.000%) - Err:2350.480225
2022-04-23 13:33:21 - fullrmc <INFO> @0 Gen:23413952 - Tr:6975579(29.792%) - Acc:20263(0.087%) - Rem:0(0.000%) - Err:2350.479736
2022-04-23 13:33:24 - fullrmc <INFO> @0 Gen:23415192 - Tr:6975983(29.793%) - Acc:20264(0.087%) - Rem:0(0.000%) - Err:2350.479736
2022-04-23 13:33:36 - fullrmc <INFO> @0 Gen:23420870 - Tr:6977648(29.792%) - Acc:20265(0.087%) - Rem:0(0.000%) - Err:2350.471191
2022-04-23 13:33:56 - fullrmc <INFO> @0 Gen:23430010 - Tr:6980391(29.793%) - Acc:202

2022-04-23 13:45:08 - fullrmc <INFO> @0 Gen:23734133 - Tr:7071448(29.794%) - Acc:20320(0.086%) - Rem:0(0.000%) - Err:2350.343262
2022-04-23 13:45:11 - fullrmc <INFO> @0 Gen:23735712 - Tr:7071923(29.794%) - Acc:20321(0.086%) - Rem:0(0.000%) - Err:2350.343262
2022-04-23 13:45:41 - fullrmc <INFO> @0 Gen:23752629 - Tr:7076929(29.794%) - Acc:20322(0.086%) - Rem:0(0.000%) - Err:2350.339355
2022-04-23 13:45:53 - fullrmc <INFO> @0 Gen:23760403 - Tr:7079213(29.794%) - Acc:20323(0.086%) - Rem:0(0.000%) - Err:2350.330811
2022-04-23 13:45:54 - fullrmc <INFO> @0 Gen:23760562 - Tr:7079265(29.794%) - Acc:20324(0.086%) - Rem:0(0.000%) - Err:2350.324219
2022-04-23 13:46:02 - fullrmc <INFO> @0 Gen:23765673 - Tr:7080816(29.794%) - Acc:20325(0.086%) - Rem:0(0.000%) - Err:2350.324219
2022-04-23 13:46:04 - fullrmc <INFO> @0 Gen:23766888 - Tr:7081171(29.794%) - Acc:20326(0.086%) - Rem:0(0.000%) - Err:2350.324219
2022-04-23 13:46:11 - fullrmc <INFO> @0 Gen:23770983 - Tr:7082380(29.794%) - Acc:20327(0.086%) - 

2022-04-23 13:52:39 - fullrmc <INFO> @0 Gen:24005188 - Tr:7152465(29.795%) - Acc:20381(0.085%) - Rem:0(0.000%) - Err:2350.203613
2022-04-23 13:52:42 - fullrmc <INFO> @0 Gen:24007353 - Tr:7153086(29.795%) - Acc:20382(0.085%) - Rem:0(0.000%) - Err:2350.203613
2022-04-23 13:52:50 - fullrmc <INFO> @0 Gen:24012248 - Tr:7154469(29.795%) - Acc:20383(0.085%) - Rem:0(0.000%) - Err:2350.203613
2022-04-23 13:52:50 - fullrmc <INFO> @0 Gen:24012266 - Tr:7154473(29.795%) - Acc:20384(0.085%) - Rem:0(0.000%) - Err:2350.163574
2022-04-23 13:52:53 - fullrmc <INFO> @0 Gen:24014138 - Tr:7155038(29.795%) - Acc:20385(0.085%) - Rem:0(0.000%) - Err:2350.163574
2022-04-23 13:53:00 - fullrmc <INFO> @0 Gen:24018529 - Tr:7156325(29.795%) - Acc:20386(0.085%) - Rem:0(0.000%) - Err:2350.163574
2022-04-23 13:53:05 - fullrmc <INFO> @0 Gen:24021555 - Tr:7157189(29.795%) - Acc:20387(0.085%) - Rem:0(0.000%) - Err:2350.163574
2022-04-23 13:53:08 - fullrmc <INFO> @0 Gen:24023472 - Tr:7157754(29.795%) - Acc:20388(0.085%) - 

2022-04-23 13:58:51 - fullrmc <INFO> @0 Gen:24231683 - Tr:7220286(29.797%) - Acc:20443(0.084%) - Rem:0(0.000%) - Err:2350.037354
2022-04-23 13:58:55 - fullrmc <INFO> @0 Gen:24234453 - Tr:7221149(29.797%) - Acc:20444(0.084%) - Rem:0(0.000%) - Err:2350.037354
2022-04-23 13:58:56 - fullrmc <INFO> @0 Gen:24234781 - Tr:7221256(29.797%) - Acc:20445(0.084%) - Rem:0(0.000%) - Err:2350.034912
2022-04-23 13:58:57 - fullrmc <INFO> @0 Gen:24235178 - Tr:7221364(29.797%) - Acc:20446(0.084%) - Rem:0(0.000%) - Err:2350.032715
2022-04-23 13:58:58 - fullrmc <INFO> @0 Gen:24235953 - Tr:7221603(29.797%) - Acc:20447(0.084%) - Rem:0(0.000%) - Err:2350.028809
2022-04-23 13:58:59 - fullrmc <INFO> @0 Gen:24236425 - Tr:7221743(29.797%) - Acc:20448(0.084%) - Rem:0(0.000%) - Err:2350.028564
2022-04-23 13:59:04 - fullrmc <INFO> @0 Gen:24239359 - Tr:7222619(29.797%) - Acc:20449(0.084%) - Rem:0(0.000%) - Err:2350.028564
2022-04-23 13:59:19 - fullrmc <INFO> @0 Gen:24249323 - Tr:7225606(29.797%) - Acc:20450(0.084%) - 

...

2022-04-23 18:03:17 - fullrmc <INFO> @0 Gen:31655893 - Tr:9436218(29.809%) - Acc:21841(0.069%) - Rem:0(0.000%) - Err:2348.437012
2022-04-23 18:03:24 - fullrmc <INFO> @0 Gen:31659336 - Tr:9437263(29.809%) - Acc:21842(0.069%) - Rem:0(0.000%) - Err:2348.436279
2022-04-23 18:03:34 - fullrmc <INFO> @0 Gen:31663957 - Tr:9438707(29.809%) - Acc:21843(0.069%) - Rem:0(0.000%) - Err:2348.435791
2022-04-23 18:03:49 - fullrmc <INFO> @0 Gen:31671465 - Tr:9440974(29.809%) - Acc:21844(0.069%) - Rem:0(0.000%) - Err:2348.426270
2022-04-23 18:03:52 - fullrmc <INFO> @0 Gen:31672984 - Tr:9441418(29.809%) - Acc:21845(0.069%) - Rem:0(0.000%) - Err:2348.424561
2022-04-23 18:04:27 - fullrmc <INFO> @0 Gen:31689307 - Tr:9446364(29.809%) - Acc:21846(0.069%) - Rem:0(0.000%) - Err:2348.423828
2022-04-23 18:04:48 - fullrmc <INFO> @0 Gen:31698721 - Tr:9449099(29.809%) - Acc:21847(0.069%) - Rem:0(0.000%) - Err:2348.423828
2022-04-23 18:04:50 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 18:04:54 - 

2022-04-23 18:16:40 - fullrmc <INFO> @0 Gen:32029985 - Tr:9548368(29.811%) - Acc:21901(0.068%) - Rem:0(0.000%) - Err:2348.368896
2022-04-23 18:16:55 - fullrmc <INFO> @0 Gen:32036426 - Tr:9550279(29.811%) - Acc:21902(0.068%) - Rem:0(0.000%) - Err:2348.368896
2022-04-23 18:17:05 - fullrmc <INFO> @0 Gen:32041370 - Tr:9551726(29.811%) - Acc:21903(0.068%) - Rem:0(0.000%) - Err:2348.367188
2022-04-23 18:17:06 - fullrmc <INFO> @0 Gen:32041744 - Tr:9551827(29.811%) - Acc:21904(0.068%) - Rem:0(0.000%) - Err:2348.367188
2022-04-23 18:17:33 - fullrmc <INFO> @0 Gen:32053443 - Tr:9555345(29.811%) - Acc:21905(0.068%) - Rem:0(0.000%) - Err:2348.367188
2022-04-23 18:18:04 - fullrmc <INFO> @0 Gen:32067553 - Tr:9559551(29.811%) - Acc:21906(0.068%) - Rem:0(0.000%) - Err:2348.367188
2022-04-23 18:18:10 - fullrmc <INFO> @0 Gen:32070327 - Tr:9560365(29.811%) - Acc:21907(0.068%) - Rem:0(0.000%) - Err:2348.367188
2022-04-23 18:18:33 - fullrmc <INFO> @0 Gen:32081098 - Tr:9563610(29.811%) - Acc:21908(0.068%) - 

2022-04-23 18:30:00 - fullrmc <INFO> @0 Gen:32339943 - Tr:9640934(29.811%) - Acc:21962(0.068%) - Rem:0(0.000%) - Err:2348.349854
2022-04-23 18:30:19 - fullrmc <INFO> @0 Gen:32347242 - Tr:9643065(29.811%) - Acc:21963(0.068%) - Rem:0(0.000%) - Err:2348.349854
2022-04-23 18:30:21 - fullrmc <INFO> @0 Gen:32348019 - Tr:9643281(29.811%) - Acc:21964(0.068%) - Rem:0(0.000%) - Err:2348.349854
2022-04-23 18:30:26 - fullrmc <INFO> @0 Gen:32349551 - Tr:9643736(29.811%) - Acc:21965(0.068%) - Rem:0(0.000%) - Err:2348.349854
2022-04-23 18:30:46 - fullrmc <INFO> @0 Gen:32357308 - Tr:9646078(29.811%) - Acc:21966(0.068%) - Rem:0(0.000%) - Err:2348.349854
2022-04-23 18:30:59 - fullrmc <INFO> @0 Gen:32362001 - Tr:9647498(29.811%) - Acc:21967(0.068%) - Rem:0(0.000%) - Err:2348.349854
2022-04-23 18:31:10 - fullrmc <INFO> @0 Gen:32366251 - Tr:9648788(29.811%) - Acc:21968(0.068%) - Rem:0(0.000%) - Err:2348.349854
2022-04-23 18:32:14 - fullrmc <INFO> @0 Gen:32390430 - Tr:9655904(29.811%) - Acc:21969(0.068%) - 

2022-04-23 18:46:54 - fullrmc <INFO> @0 Gen:32713572 - Tr:9752859(29.813%) - Acc:22022(0.067%) - Rem:0(0.000%) - Err:2348.323730
2022-04-23 18:46:59 - fullrmc <INFO> @0 Gen:32715371 - Tr:9753406(29.813%) - Acc:22023(0.067%) - Rem:0(0.000%) - Err:2348.323730
2022-04-23 18:47:12 - fullrmc <INFO> @0 Gen:32720363 - Tr:9754852(29.813%) - Acc:22024(0.067%) - Rem:0(0.000%) - Err:2348.322754
2022-04-23 18:47:39 - fullrmc <INFO> @0 Gen:32730384 - Tr:9757733(29.812%) - Acc:22025(0.067%) - Rem:0(0.000%) - Err:2348.322754
2022-04-23 18:47:41 - fullrmc <INFO> @0 Gen:32731121 - Tr:9757964(29.812%) - Acc:22026(0.067%) - Rem:0(0.000%) - Err:2348.322510
2022-04-23 18:47:57 - fullrmc <INFO> @0 Gen:32737459 - Tr:9759901(29.813%) - Acc:22027(0.067%) - Rem:0(0.000%) - Err:2348.322510
2022-04-23 18:48:01 - fullrmc <INFO> @0 Gen:32738767 - Tr:9760338(29.813%) - Acc:22028(0.067%) - Rem:0(0.000%) - Err:2348.319336
2022-04-23 18:48:12 - fullrmc <INFO> @0 Gen:32743212 - Tr:9761627(29.813%) - Acc:22029(0.067%) - 

2022-04-23 19:05:27 - fullrmc <INFO> @0 Gen:33081813 - Tr:9862933(29.814%) - Acc:22083(0.067%) - Rem:0(0.000%) - Err:2348.267822
2022-04-23 19:05:35 - fullrmc <INFO> @0 Gen:33084959 - Tr:9863898(29.814%) - Acc:22084(0.067%) - Rem:0(0.000%) - Err:2348.266357
2022-04-23 19:05:47 - fullrmc <INFO> @0 Gen:33090183 - Tr:9865486(29.814%) - Acc:22085(0.067%) - Rem:0(0.000%) - Err:2348.266357
2022-04-23 19:06:14 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 19:06:19 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 19:06:40 - fullrmc <INFO> @0 Gen:33107039 - Tr:9870415(29.814%) - Acc:22086(0.067%) - Rem:0(0.000%) - Err:2348.266357
2022-04-23 19:07:10 - fullrmc <INFO> @0 Gen:33117563 - Tr:9873548(29.814%) - Acc:22087(0.067%) - Rem:0(0.000%) - Err:2348.266357
2022-04-23 19:07:11 - fullrmc <INFO> @0 Gen:33117776 - Tr:9873615(29.814%) - Acc:22088(0.067%) - Rem:0(0.000%) - Err:2348.266357
2022-04-23 19:07:17 - fullrmc <INFO> @0 Gen:33119761 - Tr:9874218(29.814%) - Acc:220

In [21]:
ENGINE.set_groups_as_atoms()
ENGINE.run(numberOfSteps=10000000, saveFrequency=100000)

2022-04-23 19:11:22 - fullrmc <INFO> Engine @0 started 10000000 steps, total standard error is: 2348.259277
2022-04-23 19:11:25 - fullrmc <INFO> @0 Gen:33201094 - Tr:9898678(29.814%) - Acc:22105(0.067%) - Rem:0(0.000%) - Err:2348.259277
2022-04-23 19:11:46 - fullrmc <INFO> @0 Gen:33207419 - Tr:9900641(29.815%) - Acc:22106(0.067%) - Rem:0(0.000%) - Err:2348.259277
2022-04-23 19:11:49 - fullrmc <INFO> @0 Gen:33208402 - Tr:9900940(29.815%) - Acc:22107(0.067%) - Rem:0(0.000%) - Err:2348.257324
2022-04-23 19:12:26 - fullrmc <INFO> @0 Gen:33220623 - Tr:9904552(29.814%) - Acc:22108(0.067%) - Rem:0(0.000%) - Err:2348.257324
2022-04-23 19:12:30 - fullrmc <INFO> @0 Gen:33221895 - Tr:9904913(29.814%) - Acc:22109(0.067%) - Rem:0(0.000%) - Err:2348.257324
2022-04-23 19:12:46 - fullrmc <INFO> @0 Gen:33226524 - Tr:9906264(29.814%) - Acc:22110(0.067%) - Rem:0(0.000%) - Err:2348.257324
2022-04-23 19:13:17 - fullrmc <INFO> @0 Gen:33234742 - Tr:9908719(29.814%) - Acc:22111(0.067%) - Rem:0(0.000%) - Err:2

2022-04-23 19:28:26 - fullrmc <INFO> @0 Gen:33541295 - Tr:10000362(29.815%) - Acc:22165(0.066%) - Rem:0(0.000%) - Err:2348.190918
2022-04-23 19:28:46 - fullrmc <INFO> @0 Gen:33548066 - Tr:10002375(29.815%) - Acc:22166(0.066%) - Rem:0(0.000%) - Err:2348.190918
2022-04-23 19:28:49 - fullrmc <INFO> @0 Gen:33548933 - Tr:10002644(29.815%) - Acc:22167(0.066%) - Rem:0(0.000%) - Err:2348.190918
2022-04-23 19:29:15 - fullrmc <INFO> @0 Gen:33557493 - Tr:10005204(29.815%) - Acc:22168(0.066%) - Rem:0(0.000%) - Err:2348.190918
2022-04-23 19:29:26 - fullrmc <INFO> @0 Gen:33560960 - Tr:10006290(29.815%) - Acc:22169(0.066%) - Rem:0(0.000%) - Err:2348.185547
2022-04-23 19:29:30 - fullrmc <INFO> @0 Gen:33562261 - Tr:10006664(29.815%) - Acc:22170(0.066%) - Rem:0(0.000%) - Err:2348.185059
2022-04-23 19:29:34 - fullrmc <INFO> @0 Gen:33563942 - Tr:10007133(29.815%) - Acc:22171(0.066%) - Rem:0(0.000%) - Err:2348.180664
2022-04-23 19:30:01 - fullrmc <INFO> @0 Gen:33572979 - Tr:10009854(29.815%) - Acc:22172(0.

2022-04-23 19:46:27 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-23 19:46:31 - fullrmc <INFO> @0 Runtime save is successful
2022-04-23 19:46:32 - fullrmc <INFO> @0 Gen:33900228 - Tr:10107384(29.815%) - Acc:22225(0.066%) - Rem:0(0.000%) - Err:2348.107178
2022-04-23 19:46:34 - fullrmc <INFO> @0 Gen:33900957 - Tr:10107589(29.815%) - Acc:22226(0.066%) - Rem:0(0.000%) - Err:2348.107178
2022-04-23 19:46:56 - fullrmc <INFO> @0 Gen:33905469 - Tr:10108904(29.815%) - Acc:22227(0.066%) - Rem:0(0.000%) - Err:2348.107178
2022-04-23 19:46:57 - fullrmc <INFO> @0 Gen:33905826 - Tr:10109007(29.815%) - Acc:22228(0.066%) - Rem:0(0.000%) - Err:2348.107178
2022-04-23 19:47:12 - fullrmc <INFO> @0 Gen:33910745 - Tr:10110452(29.815%) - Acc:22229(0.066%) - Rem:0(0.000%) - Err:2348.107178
2022-04-23 19:47:17 - fullrmc <INFO> @0 Gen:33912769 - Tr:10111065(29.815%) - Acc:22230(0.066%) - Rem:0(0.000%) - Err:2348.107178
2022-04-23 19:47:32 - fullrmc <INFO> @0 Gen:33917476 - Tr:10112478(29.815%) - 

2022-04-23 20:03:18 - fullrmc <INFO> @0 Gen:34211502 - Tr:10200230(29.815%) - Acc:22284(0.065%) - Rem:0(0.000%) - Err:2348.060059
2022-04-23 20:03:41 - fullrmc <INFO> @0 Gen:34219769 - Tr:10202714(29.815%) - Acc:22285(0.065%) - Rem:0(0.000%) - Err:2348.060059
2022-04-23 20:03:54 - fullrmc <INFO> @0 Gen:34224126 - Tr:10204039(29.815%) - Acc:22286(0.065%) - Rem:0(0.000%) - Err:2348.060059
2022-04-23 20:04:10 - fullrmc <INFO> @0 Gen:34229555 - Tr:10205635(29.815%) - Acc:22287(0.065%) - Rem:0(0.000%) - Err:2348.060059
2022-04-23 20:04:10 - fullrmc <INFO> @0 Gen:34229734 - Tr:10205691(29.815%) - Acc:22288(0.065%) - Rem:0(0.000%) - Err:2348.060059
2022-04-23 20:04:17 - fullrmc <INFO> @0 Gen:34231879 - Tr:10206312(29.815%) - Acc:22289(0.065%) - Rem:0(0.000%) - Err:2348.060059
2022-04-23 20:04:17 - fullrmc <INFO> @0 Gen:34232127 - Tr:10206381(29.815%) - Acc:22290(0.065%) - Rem:0(0.000%) - Err:2348.060059
2022-04-23 20:04:36 - fullrmc <INFO> @0 Gen:34238542 - Tr:10208283(29.815%) - Acc:22291(0.

2022-04-23 20:14:23 - fullrmc <INFO> @0 Gen:34442611 - Tr:10269074(29.815%) - Acc:22345(0.065%) - Rem:0(0.000%) - Err:2347.924561
2022-04-23 20:14:26 - fullrmc <INFO> @0 Gen:34443838 - Tr:10269420(29.815%) - Acc:22346(0.065%) - Rem:0(0.000%) - Err:2347.924561
2022-04-23 20:14:44 - fullrmc <INFO> @0 Gen:34450268 - Tr:10271339(29.815%) - Acc:22347(0.065%) - Rem:0(0.000%) - Err:2347.918945
2022-04-23 20:15:01 - fullrmc <INFO> @0 Gen:34455565 - Tr:10272918(29.815%) - Acc:22348(0.065%) - Rem:0(0.000%) - Err:2347.918945
2022-04-23 20:15:15 - fullrmc <INFO> @0 Gen:34459838 - Tr:10274227(29.815%) - Acc:22349(0.065%) - Rem:0(0.000%) - Err:2347.901123
2022-04-23 20:15:20 - fullrmc <INFO> @0 Gen:34461622 - Tr:10274765(29.815%) - Acc:22350(0.065%) - Rem:0(0.000%) - Err:2347.901123
2022-04-23 20:15:35 - fullrmc <INFO> @0 Gen:34466884 - Tr:10276340(29.815%) - Acc:22351(0.065%) - Rem:0(0.000%) - Err:2347.900391
2022-04-23 20:15:43 - fullrmc <INFO> @0 Gen:34469467 - Tr:10277116(29.815%) - Acc:22352(0.

...

2022-04-24 00:55:20 - fullrmc <INFO> @0 Gen:41990888 - Tr:12522490(29.822%) - Acc:23908(0.057%) - Rem:0(0.000%) - Err:2344.545166
2022-04-24 00:55:24 - fullrmc <INFO> @0 Gen:41992582 - Tr:12522966(29.822%) - Acc:23909(0.057%) - Rem:0(0.000%) - Err:2344.541992
2022-04-24 00:55:41 - fullrmc <INFO> @0 Gen:41997514 - Tr:12524404(29.822%) - Acc:23910(0.057%) - Rem:0(0.000%) - Err:2344.541504
2022-04-24 00:55:48 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-24 00:55:51 - fullrmc <INFO> @0 Runtime save is successful
2022-04-24 00:55:52 - fullrmc <INFO> @0 Gen:42000093 - Tr:12525158(29.822%) - Acc:23911(0.057%) - Rem:0(0.000%) - Err:2344.541504
2022-04-24 00:56:05 - fullrmc <INFO> @0 Gen:42005705 - Tr:12526779(29.822%) - Acc:23912(0.057%) - Rem:0(0.000%) - Err:2344.541504
2022-04-24 00:56:13 - fullrmc <INFO> @0 Gen:42008777 - Tr:12527710(29.822%) - Acc:23913(0.057%) - Rem:0(0.000%) - Err:2344.540527
2022-04-24 00:56:34 - fullrmc <INFO> @0 Gen:42017709 - Tr:12530337(29.822%) - 

2022-04-24 01:06:11 - fullrmc <INFO> @0 Gen:42254175 - Tr:12600882(29.822%) - Acc:23968(0.057%) - Rem:0(0.000%) - Err:2344.498779
2022-04-24 01:06:20 - fullrmc <INFO> @0 Gen:42257619 - Tr:12601904(29.822%) - Acc:23969(0.057%) - Rem:0(0.000%) - Err:2344.498779
2022-04-24 01:06:21 - fullrmc <INFO> @0 Gen:42257962 - Tr:12602006(29.822%) - Acc:23970(0.057%) - Rem:0(0.000%) - Err:2344.498779
2022-04-24 01:06:34 - fullrmc <INFO> @0 Gen:42263651 - Tr:12603730(29.822%) - Acc:23971(0.057%) - Rem:0(0.000%) - Err:2344.498779
2022-04-24 01:06:36 - fullrmc <INFO> @0 Gen:42264442 - Tr:12603944(29.822%) - Acc:23972(0.057%) - Rem:0(0.000%) - Err:2344.494629
2022-04-24 01:06:38 - fullrmc <INFO> @0 Gen:42265233 - Tr:12604179(29.822%) - Acc:23973(0.057%) - Rem:0(0.000%) - Err:2344.492188
2022-04-24 01:07:09 - fullrmc <INFO> @0 Gen:42278495 - Tr:12608101(29.822%) - Acc:23974(0.057%) - Rem:0(0.000%) - Err:2344.492188
2022-04-24 01:07:20 - fullrmc <INFO> @0 Gen:42283035 - Tr:12609435(29.821%) - Acc:23975(0.

2022-04-24 01:18:58 - fullrmc <INFO> @0 Gen:42566929 - Tr:12694616(29.823%) - Acc:24028(0.056%) - Rem:0(0.000%) - Err:2344.470215
2022-04-24 01:19:01 - fullrmc <INFO> @0 Gen:42568380 - Tr:12695041(29.823%) - Acc:24029(0.056%) - Rem:0(0.000%) - Err:2344.470215
2022-04-24 01:19:22 - fullrmc <INFO> @0 Gen:42577762 - Tr:12697927(29.823%) - Acc:24030(0.056%) - Rem:0(0.000%) - Err:2344.470215
2022-04-24 01:19:24 - fullrmc <INFO> @0 Gen:42578630 - Tr:12698182(29.823%) - Acc:24031(0.056%) - Rem:0(0.000%) - Err:2344.470215
2022-04-24 01:19:47 - fullrmc <INFO> @0 Gen:42590308 - Tr:12701671(29.823%) - Acc:24032(0.056%) - Rem:0(0.000%) - Err:2344.470215
2022-04-24 01:20:00 - fullrmc <INFO> @0 Gen:42596034 - Tr:12703388(29.823%) - Acc:24033(0.056%) - Rem:0(0.000%) - Err:2344.470215
2022-04-24 01:20:00 - fullrmc <INFO> @0 Gen:42596322 - Tr:12703485(29.823%) - Acc:24034(0.056%) - Rem:0(0.000%) - Err:2344.470215
2022-04-24 01:20:01 - fullrmc <INFO> @0 Gen:42596478 - Tr:12703530(29.823%) - Acc:24035(0.

2022-04-24 01:29:16 - fullrmc <INFO> @0 Gen:42875980 - Tr:12786615(29.822%) - Acc:24088(0.056%) - Rem:0(0.000%) - Err:2344.297363
2022-04-24 01:29:21 - fullrmc <INFO> @0 Gen:42879405 - Tr:12787611(29.822%) - Acc:24089(0.056%) - Rem:0(0.000%) - Err:2344.295410
2022-04-24 01:29:24 - fullrmc <INFO> @0 Gen:42881236 - Tr:12788145(29.822%) - Acc:24090(0.056%) - Rem:0(0.000%) - Err:2344.295410
2022-04-24 01:29:26 - fullrmc <INFO> @0 Gen:42882632 - Tr:12788540(29.822%) - Acc:24091(0.056%) - Rem:0(0.000%) - Err:2344.291260
2022-04-24 01:29:28 - fullrmc <INFO> @0 Gen:42883521 - Tr:12788808(29.822%) - Acc:24092(0.056%) - Rem:0(0.000%) - Err:2344.291260
2022-04-24 01:29:33 - fullrmc <INFO> @0 Gen:42886811 - Tr:12789767(29.822%) - Acc:24093(0.056%) - Rem:0(0.000%) - Err:2344.289551
2022-04-24 01:29:53 - fullrmc <INFO> @0 Runtime saving ... DON'T INTERRUPT
2022-04-24 01:29:57 - fullrmc <INFO> @0 Runtime save is successful
2022-04-24 01:30:04 - fullrmc <INFO> @0 Gen:42904633 - Tr:12794934(29.822%) - 

2022-04-24 01:34:39 - fullrmc <INFO> @0 Gen:43076755 - Tr:12846782(29.823%) - Acc:24149(0.056%) - Rem:0(0.000%) - Err:2344.055908
2022-04-24 01:34:46 - fullrmc <INFO> @0 Gen:43080896 - Tr:12848011(29.823%) - Acc:24150(0.056%) - Rem:0(0.000%) - Err:2344.045166
2022-04-24 01:34:50 - fullrmc <INFO> @0 Gen:43083299 - Tr:12848775(29.823%) - Acc:24151(0.056%) - Rem:0(0.000%) - Err:2344.030518
2022-04-24 01:34:57 - fullrmc <INFO> @0 Gen:43087689 - Tr:12850090(29.823%) - Acc:24152(0.056%) - Rem:0(0.000%) - Err:2344.026855
2022-04-24 01:34:59 - fullrmc <INFO> @0 Gen:43088892 - Tr:12850463(29.823%) - Acc:24153(0.056%) - Rem:0(0.000%) - Err:2344.024658
2022-04-24 01:35:05 - fullrmc <INFO> @0 Gen:43093008 - Tr:12851698(29.823%) - Acc:24154(0.056%) - Rem:0(0.000%) - Err:2344.024658
2022-04-24 01:35:09 - fullrmc <INFO> @0 Gen:43095103 - Tr:12852300(29.823%) - Acc:24155(0.056%) - Rem:0(0.000%) - Err:2343.982422
2022-04-24 01:35:10 - fullrmc <INFO> @0 Gen:43095877 - Tr:12852550(29.823%) - Acc:24156(0.